# Capstone Project: NHL Cap Hit Predictions

#### Brian Johns, April 2022.  BrainStation

## Introduction

It is the goal of every team in the National Hockey League (NHL) to build a team that will eventually win the Stanley Cup.  However, the business part of the league restricts how money is spread to the players in order to maintain competitive balance across the league.  For the 2021-22 season, the main restrictions are:

- All teams must have **20-23 active players** on the roster.

- No team is allowed to spend more than **$81.5 million total** on their active players.  This is the Salary Cap for each team.

- The maximum salary for any player is **$16.3 million**

- For rookies, they sign an 'entry-level' contract after they are drafted.  The maximum entry-level contract is **$925,000** for the first 3 years of their contract.

- The minimum salary for any player is **$700,000**.

- The amount of money a player counts against the Salary Cap for their team is determined by the **average salary across the full duration of their contract**.  This is a player's 'Cap Hit'.

In order to evaluate talent within the league, the use of advanced analytics has become more popular in the last 10 years.  The amount of statistical information about the players in the league is immense, giving a team who can effectively use this data an advantage in player acquisition and performance.

As the information about players has changed and grown, I am curious to see if it has had an effect on team building strategies in the NHL.  The question I would like to answer is:

**How can we predict and evaluate the Cap Hit of NHL players using their basic and advanced statistics?**

If I am able to do this, I would be able to:

1. Identify statistics that players are currently being rewarded for.

2. Identify statistics that may be undervalued and could be gaps in the market for players.

3. Identify players that are under/overvalued based on their performance profile.

## Notebook #1: Data Cleaning

In order to answer our business question, we first must collect and clean the data to be used for analysis.

I will be retieving data from two primary sources:

1. Financial Statistics are from [CapFriendly](https://www.capfriendly.com/).

    - The information from CapFriendly has to be scraped across multiple pages from multiple years in order to get all of the financial information needed.
    

2. Player Performance Statistics from [Evolving-Hockey](https://evolving-hockey.com/).

    - The only statistics that are free to view on Evolving Hockey are the basic statistics which can be found here: [Skater Tables](https://evolving-hockey.com/stats/skater_standard/)

The more advanced statistics, and the ability to download the information, requires a subscription to the site.  I had a previous subscription so I have downloaded all of the data into 8 separate csv files.

From here, the goals for this notebook will be to:
1. [Data Wrangling](#wrangling) - Effectively upload the data from each source into separate dataframes
2. [Merging Data](#merging) - Do some initial data cleaning and formatting in order to merge the separate dataframes together
3. [Data Cleaning](#cleaning) - After merging the data, finish cleaning the data in order to maximize the amount of data available for analysis

### Data Wrangling

<a id = 'wrangling'></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import requests
import glob

In [2]:
#This dataset will have a lot of columns, so we'll set up our options to view them all here
pd.set_option("display.max_columns", None)

In [3]:
#First we are going to scrape the CapFriendly website to get the financial details for every player since the 2007-8
#season
money_df = []
for year in range (2008, 2023): #This will gather the last 15 years of data
    for page in range (1,35):
        #The salary information is spread across multiple pages, 50 players per page
        r = requests.get(f'https://www.capfriendly.com/browse/active/{year}?pg={page}').content
        table = pd.read_html(r)
        
        #Only one table is positioned on each page
        df = table[0]
        
        #I am manually inputting the year the data is scraped from as it is not actually listed on the page
        df['Year'] = year
        money_df.append(df)
        
money_df = pd.concat(money_df, ignore_index = True)

In [4]:
money_df.head()

,PLAYER,TEAM,AGE,POS,HANDED,GP,G,A,P,P/GP,+/-,Sh,Sh%,TOI,W,L,SO,GAA,Sv%,CLAUSE,EXPIRY,CAP HIT,SALARY,Year
0,1. Jaromír Jágr,-,35,RW,Left,0,0,0,0,0.00,0,0,0.00,NaN,-,-,-,-,-,NaN,UFA,"$7,920,000","$8,360,000",2008
1,2. Brad Richards,-,27,"C, LW",Left,0,0,0,0,0.00,0,0,0.00,NaN,-,-,-,-,-,NTC,UFA,"$7,800,000","$7,800,000",2008
2,3. Nicklas Lidström,-,37,D,Left,0,0,0,0,0.00,0,0,0.00,NaN,-,-,-,-,-,NMC,UFA,"$7,600,000","$7,600,000",2008
3,4. Zdeno Chara,NYI,30,LD,Left,0,0,0,0,0.00,0,0,0.00,NaN,-,-,-,-,-,NTC,UFA,"$7,500,000","$7,500,000",2008
4,5. Scott Gomez,-,27,C,Left,0,0,0,0,0.00,0,0,0.00,NaN,-,-,-,-,-,NaN,UFA,"$7,357,143","$10,000,000",2008


It looks like there is some statistical data available here but it is only filled for *some* of the players.  Fortunately, it looks like the `CAP HIT` and `SALARY` columns are filled appropriately, which is the primary use of this source.

However, the financial columns are not formatted appropriately to be used as a number, so it will need to be modfied prior to analysis.  Also, there is an issue with the `PLAYER` category, which includes the number from the index on the website (which will have to be split out).  Some names also include accents in some of the names, so we will have to standardize the formatting in order to merge the data.

In [5]:
money_df.shape

(19439, 24)

In [6]:
#All objects except for the year that I manually inputed
#We have already seen the Cap Hit and Salary columns need to be changed to numbers.
#Much of this data will be repeated by what we get from the Evolving Hockey dataset and will be dropped from this one
money_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19439 entries, 0 to 19438
Data columns (total 24 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   PLAYER   19439 non-null  object
 1   TEAM     19439 non-null  object
 2   AGE      19439 non-null  object
 3   POS      19439 non-null  object
 4   HANDED   19439 non-null  object
 5   GP       19439 non-null  object
 6   G        19439 non-null  object
 7   A        19439 non-null  object
 8   P        19439 non-null  object
 9   P/GP     19439 non-null  object
 10  +/-      19439 non-null  object
 11  Sh       19439 non-null  object
 12  Sh%      19439 non-null  object
 13  TOI      11626 non-null  object
 14  W        19439 non-null  object
 15  L        19439 non-null  object
 16  SO       19439 non-null  object
 17  GAA      19439 non-null  object
 18  Sv%      19439 non-null  object
 19  CLAUSE   2074 non-null   object
 20  EXPIRY   19364 non-null  object
 21  CAP HIT  19439 non-null  object
 22

Much of the data for the 19439 rows may not be included in this project.  I will only be looking at Skaters and not goalies, so a large portion of this data will be excluded.  As well, depending on the player statistics and overall relevancy of the data, I will likely only include data from the last 10 years, so data gathered from before that may not be included.

In [7]:
money_df.duplicated().sum()

0

In [8]:
money_df.isna().sum()

PLAYER         0
TEAM           0
AGE            0
POS            0
HANDED         0
GP             0
G              0
A              0
P              0
P/GP           0
+/-            0
Sh             0
Sh%            0
TOI         7813
W              0
L              0
SO             0
GAA            0
Sv%            0
CLAUSE     17365
EXPIRY        75
CAP HIT        0
SALARY         0
Year           0
dtype: int64

`TOI` (time on ice), will have much more detail from the Evolving Hockey datasets so I do not need to keep it from the CapFriendly data.  `CLAUSE` (ie. No-Trade Clause) will be irrelevant for our dataset as well.

Next I will take the accents out of the players names.

In [9]:
money_df['PLAYER'] = money_df['PLAYER'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

For the Evolving Hockey data, their advanced statistics are only available to paid subscribers of the website.  As a previous subscriber, I have downloaded all of the relevant data and placed it into a data folder.  These files will be made available with the submission of this project.

In [10]:
#This is everything in the folder, so the only items should be the Evolving Hockey data provided
file_paths = glob.glob("data/eh/*")

eh_df = pd.DataFrame()

for file in file_paths:
    counter = 0
    df = pd.read_csv(file)
    
    #I will format the names as they come in to remove accents
    df['Player'] = df['Player'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
    counter += 1
    if eh_df.shape[0] == 0:
        eh_df = pd.concat([eh_df, df])
    else:
        eh_df = eh_df.merge(df,
                            #Fortunately, Evolving Hockey has created a unique ID for each player that is on every file
                            left_on=['Player', 'API ID', 'Season'], 
                            right_on=['Player', 'API ID', 'Season'], 
                            how='left', 
                            suffixes = ('', counter))

In [11]:
eh_df.head()

,Player,EH_ID,API ID,Season,Team,Position,Shoots,Birthday,Age,Draft Yr,Draft Rd,Draft Ov,GP,TOI,GF%,SF%,FF%,CF%,xGF%,GF/60,GA/60,SF/60,SA/60,FF/60,FA/60,CF/60,CA/60,xGF/60,xGA/60,G±/60,S±/60,F±/60,C±/60,xG±/60,Sh%,Sv%,EH_ID1,Team1,Position1,Shoots1,Birthday1,Age1,Draft Yr1,Draft Rd1,Draft Ov1,GP1,xEVO_GAR/60,xEVD_GAR/60,xPPO_GAR/60,xSHD_GAR/60,xTake_GAR/60,xDraw_GAR/60,xOff_GAR/60,xDef_GAR/60,xPens_GAR/60,xGAR/60,xWAR/60,xSPAR/60,EH_ID1,Team1,Position1,Shoots1,Birthday1,Age1,Draft Yr1,Draft Rd1,Draft Ov1,GP1,xEVO_GAR,xEVD_GAR,xPPO_GAR,xSHD_GAR,xTake_GAR,xDraw_GAR,xOff_GAR,xDef_GAR,xPens_GAR,xGAR,xWAR,xSPAR,EH_ID1,Team1,Position1,Shoots1,Birthday1,Age1,Draft Yr1,Draft Rd1,Draft Ov1,GP1,TOI1,G/60,A1/60,A2/60,Points/60,iSF/60,iFF/60,iCF/60,ixG/60,Sh%1,FSh%,xFSh%,iBLK/60,GIVE/60,TAKE/60,iHF/60,iHA/60,iPENT2/60,iPEND2/60,iPENT5/60,iPEND5/60,iPEN±/60,FOW/60,FOL/60,FO±/60,EH_ID1,Team1,Position1,Shoots1,Birthday1,Age1,Draft Yr1,Draft Rd1,Draft Ov1,GP1,TOI1,G,A1,A2,Points,iSF,iFF,iCF,ixG,Sh%1,FSh%1,xFSh%1,iBLK,GIVE,TAKE,iHF,iHA,iPENT2,iPEND2,iPENT5,iPEND5,iPEN±,FOW,FOL,FO±,EH_ID1,Team1,Position1,Shoots1,Birthday1,Age1,Draft Yr1,Draft Rd1,Draft Ov1,GP1,TOI1,GF%1,SF%1,FF%1,CF%1,xGF%1,GF,GA,SF,SA,FF,FA,CF,CA,xGF,xGA,G±,S±,F±,C±,xG±,Sh%1,Sv%1,EH_ID1,Team1,Position1,Shoots1,Birthday1,Age1,Draft Yr1,Draft Rd1,Draft Ov1,GP1,TOI_All,TOI_EV,TOI_PP,TOI_SH,EVO_GAR,EVD_GAR,PPO_GAR,SHD_GAR,Take_GAR,Draw_GAR,Off_GAR,Def_GAR,Pens_GAR,GAR,WAR,SPAR,EH_ID1,Team1,Position1,Shoots1,Birthday1,Age1,Draft Yr1,Draft Rd1,Draft Ov1,GP1,EVO_GAR/60,EVD_GAR/60,PPO_GAR/60,SHD_GAR/60,Take_GAR/60,Draw_GAR/60,Off_GAR/60,Def_GAR/60,Pens_GAR/60,GAR/60,WAR/60,SPAR/60
0,A.J. Greer,A.J..GREER,8478421,16-17,COL,L,L,1996-12-14,19,2015,2.0,39.0,5,68.28,24.40,51.38,56.00,51.91,64.27,0.89,2.75,30.50,28.87,43.54,34.21,55.68,51.58,2.75,1.53,-1.86,1.63,9.33,4.09,1.22,2.91,90.47,A.J..GREER,COL,L,L,1996-12-14,19,2015,2.0,39.0,5,0.547,0.776,-0.820,0.584,0.135,-0.125,0.488,0.772,0.011,1.223,0.230,0.446,A.J..GREER,COL,L,L,1996-12-14,19,2015,2.0,39.0,5,0.6,0.8,0.0,0.0,0.2,-0.1,0.5,0.8,0.0,1.4,0.3,0.5,A.J..GREER,COL,L,L,1996-12-14,19,2015,2.0,39.0,5,68.28,0.00,0.00,0.88,0.88,7.03,8.79,9.67,0.90,0.00,0.00,10.20,2.64,1.76,0.88,3.51,5.27,1.76,1.76,0.00,0.00,0.00,0.00,0.00,0.00,A.J..GREER,COL,L,L,1996-12-14,19,2015,2.0,39.0,5,68.28,0,0,1,1,8,10,11,1.02,0.00,0.00,10.20,3,2,1,4,6,2,2,0,0,0,0,0,0,A.J..GREER,COL,L,L,1996-12-14,19,2015,2.0,39.0,5,68.28,24.40,51.38,56.00,51.91,64.27,1.01,3.13,34.71,32.85,49.55,38.93,63.36,58.70,3.13,1.74,-2.12,1.86,10.62,4.66,1.39,2.91,90.47,A.J..GREER,COL,L,L,1996-12-14,19,2015,2.0,39.0,5,68.3,63.8,2.9,1.3,-0.5,0.6,0.0,0.0,0.2,-0.1,-0.6,0.6,0.0,0.1,0.0,0.0,A.J..GREER,COL,L,L,1996-12-14,19,2015,2.0,39.0,5,-0.491,0.578,-0.792,0.062,0.135,-0.125,-0.504,0.568,0.011,0.060,0.011,0.022
1,A.J. Greer,A.J..GREER,8478421,17-18,COL,L,L,1996-12-14,20,2015,2.0,39.0,17,125.68,67.51,41.38,39.94,40.27,39.82,1.91,0.92,24.45,34.65,31.58,47.50,41.32,61.30,1.69,2.55,0.99,-10.19,-15.92,-19.98,-0.86,7.83,97.34,A.J..GREER,COL,L,L,1996-12-14,20,2015,2.0,39.0,17,-0.351,-0.404,-0.068,0.000,-0.122,0.047,-0.345,-0.404,-0.075,-0.810,-0.158,-0.302,A.J..GREER,COL,L,L,1996-12-14,20,2015,2.0,39.0,17,-0.7,-0.8,0.0,0.0,-0.3,0.1,-0.7,-0.8,-0.2,-1.7,-0.3,-0.6,A.J..GREER,COL,L,L,1996-12-14,20,2015,2.0,39.0,17,125.68,0.00,0.95,0.48,1.43,6.21,7.16,9.55,0.32,0.00,0.00,4.40,2.39,0.95,1.43,14.32,7.64,3.82,2.39,0.48,0.48,-1.43,0.48,0.95,-0.48,A.J..GREER,COL,L,L,1996-12-14,20,2015,2.0,39.0,17,125.68,0,2,1,3,13,15,20,0.66,0.00,0.00,4.40,5,2,3,30,16,8,5,1,1,-3,1,2,-1,A.J..GREER,COL,L,L,1996-12-14,20,2015,2.0,39.0,17,125.68,67.51,41.38,39.94,40.27,39.82,4.01,1.93,51.22,72.57,66.15,99.49,86.56,128.41,3.54,5.35,2.08,-21.35,-33.34,-41.85,-1.81,7.83,97.34,A.J..GREER,COL,L,L,1996-12-14,20,2015,2.0,39.0,17,125.7,122.3,2.8,0.0,0.0,-0.6,0.0,0.0,-0.3,0.1,-0.1,-0.6,-0.2,-0.8,-0.2,-0.3,A.J..GREER,COL,L,L,1996-12-14,20,2015,2.0,39.0,17,-0.022,-0.301,-0.120,0.000,-0.122,0.047,-0.024,-0.301,-0.075,-0.392,-0.076,-0.146
2,A.J.

In [12]:
eh_df.shape

(13334, 231)

Looking through these statistics, there are many stats that were in more than one files.  I will drop all of the columns that are duplicated.

In [13]:
eh_df = eh_df.T.drop_duplicates().T

In [14]:
eh_df.shape

(13334, 147)

There are two columns that still have the same name when they merged:

1. Position and Position1
2. Sh% and Sh%1

We'll have a closer look at them both to see if the data itself is duplicative.

In [15]:
eh_df.head()

,Player,EH_ID,API ID,Season,Team,Position,Shoots,Birthday,Age,Draft Yr,Draft Rd,Draft Ov,GP,TOI,GF%,SF%,FF%,CF%,xGF%,GF/60,GA/60,SF/60,SA/60,FF/60,FA/60,CF/60,CA/60,xGF/60,xGA/60,G±/60,S±/60,F±/60,C±/60,xG±/60,Sh%,Sv%,Position1,xEVO_GAR/60,xEVD_GAR/60,xPPO_GAR/60,xSHD_GAR/60,xTake_GAR/60,xDraw_GAR/60,xOff_GAR/60,xDef_GAR/60,xPens_GAR/60,xGAR/60,xWAR/60,xSPAR/60,xEVO_GAR,xEVD_GAR,xPPO_GAR,xSHD_GAR,xTake_GAR,xDraw_GAR,xOff_GAR,xDef_GAR,xPens_GAR,xGAR,xWAR,xSPAR,G/60,A1/60,A2/60,Points/60,iSF/60,iFF/60,iCF/60,ixG/60,Sh%1,FSh%,xFSh%,iBLK/60,GIVE/60,TAKE/60,iHF/60,iHA/60,iPENT2/60,iPEND2/60,iPENT5/60,iPEND5/60,iPEN±/60,FOW/60,FOL/60,FO±/60,G,A1,A2,Points,iSF,iFF,iCF,ixG,iBLK,GIVE,TAKE,iHF,iHA,iPENT2,iPEND2,iPENT5,iPEND5,iPEN±,FOW,FOL,FO±,GF,GA,SF,SA,FF,FA,CF,CA,xGF,xGA,G±,S±,F±,C±,xG±,TOI_All,TOI_EV,TOI_PP,TOI_SH,EVO_GAR,EVD_GAR,PPO_GAR,SHD_GAR,Take_GAR,Off_GAR,Def_GAR,GAR,WAR,SPAR,EVO_GAR/60,EVD_GAR/60,PPO_GAR/60,SHD_GAR/60,Take_GAR/60,Draw_GAR/60,Off_GAR/60,Def_GAR/60,Pens_GAR/60,GAR/60,WAR/60,SPAR/60
0,A.J. Greer,A.J..GREER,8478421,16-17,COL,L,L,1996-12-14,19,2015,2.0,39.0,5,68.28,24.4,51.38,56.0,51.91,64.27,0.89,2.75,30.5,28.87,43.54,34.21,55.68,51.58,2.75,1.53,-1.86,1.63,9.33,4.09,1.22,2.91,90.47,L,0.547,0.776,-0.82,0.584,0.135,-0.125,0.488,0.772,0.011,1.223,0.23,0.446,0.6,0.8,0.0,0.0,0.2,-0.1,0.5,0.8,0.0,1.4,0.3,0.5,0.0,0.0,0.88,0.88,7.03,8.79,9.67,0.9,0.0,0.0,10.2,2.64,1.76,0.88,3.51,5.27,1.76,1.76,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,1,8,10,11,1.02,3,2,1,4,6,2,2,0,0,0,0,0,0,1.01,3.13,34.71,32.85,49.55,38.93,63.36,58.7,3.13,1.74,-2.12,1.86,10.62,4.66,1.39,68.3,63.8,2.9,1.3,-0.5,0.6,0.0,0.0,0.2,-0.6,0.6,0.1,0.0,0.0,-0.491,0.578,-0.792,0.062,0.135,-0.125,-0.504,0.568,0.011,0.06,0.011,0.022
1,A.J. Greer,A.J..GREER,8478421,17-18,COL,L,L,1996-12-14,20,2015,2.0,39.0,17,125.68,67.51,41.38,39.94,40.27,39.82,1.91,0.92,24.45,34.65,31.58,47.5,41.32,61.3,1.69,2.55,0.99,-10.19,-15.92,-19.98,-0.86,7.83,97.34,L,-0.351,-0.404,-0.068,0.0,-0.122,0.047,-0.345,-0.404,-0.075,-0.81,-0.158,-0.302,-0.7,-0.8,0.0,0.0,-0.3,0.1,-0.7,-0.8,-0.2,-1.7,-0.3,-0.6,0.0,0.95,0.48,1.43,6.21,7.16,9.55,0.32,0.0,0.0,4.4,2.39,0.95,1.43,14.32,7.64,3.82,2.39,0.48,0.48,-1.43,0.48,0.95,-0.48,0,2,1,3,13,15,20,0.66,5,2,3,30,16,8,5,1,1,-3,1,2,-1,4.01,1.93,51.22,72.57,66.15,99.49,86.56,128.41,3.54,5.35,2.08,-21.35,-33.34,-41.85,-1.81,125.7,122.3,2.8,0.0,0.0,-0.6,0.0,0.0,-0.3,-0.1,-0.6,-0.8,-0.2,-0.3,-0.022,-0.301,-0.12,0.0,-0.122,0.047,-0.024,-0.301,-0.075,-0.392,-0.076,-0.146
2,A.J. Greer,A.J..GREER,8478421,18-19,COL,L,L,1996-12-14,21,2015,2.0,39.0,15,94.02,33.44,49.06,50.64,48.87,47.84,1.31,2.6,28.59,29.68,38.85,37.87,49.32,51.59,2.19,2.39,-1.3,-1.09,0.98,-2.27,-0.2,4.58,91.23,L,-0.067,0.279,0.0,0.0,-0.317,-0.124,-0.067,0.279,-0.442,-0.23,-0.043,-0.081,-0.1,0.4,0.0,0.0,-0.5,-0.2,-0.1,0.4,-0.7,-0.4,-0.1,-0.1,0.64,0.64,0.0,1.28,5.74,7.02,8.93,0.8,11.11,9.09,11.45,3.19,0.64,2.55,12.76,12.76,4.47,1.28,0.0,0.0,-3.19,1.28,1.91,-0.64,1,1,0,2,9,11,14,1.26,5,1,4,20,20,7,2,0,0,-5,2,3,-1,2.05,4.08,44.8,46.51,60.88,59.34,77.28,80.84,3.43,3.74,-2.03,-1.71,1.54,-3.56,-0.31,94.0,93.8,0.0,0.0,-0.8,-0.2,0.0,0.0,-0.5,-0.8,-0.2,-1.7,-0.3,-0.6,-0.539,-0.116,0.0,0.0,-0.317,-0.124,-0.539,-0.116,-0.442,-1.095,-0.204,-0.384
3,A.J. Greer,A.J..GREER,8478421,20-21,N.J,L,L,1996-12-14,23,2015,2.0,39.0,1,8.55,0.0,7.41,21.06,18.63,15.45,0.0,0.0,6.46,80.7,25.54,95.72,25.26,110.32,1.33,7.3,0.0,-74.25,-70.18,-85.05,-5.96,0.0,100.0,L,0.206,-0.791,0.0,0.0,-1.372,-0.12,0.206,-0.791,-1.492,-2.078,-0.373,-0.695,0.0,-0.1,0.0,0.0,-0.2,0.0,0.0,-0.1,-0.2,-0.3,-0.1,-0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.02,7.02,0.0,7.02,7.02,-7.02,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0,0,0,0,0,1,1,0,1,1,-1,0,0,0,0.0,0.0,0.92,11.5,3.64,13.64,3.6,15.72,0.19,1.04,0.0,-10.58,-10.0,-12.12,-0.85,8.6,8.6,0.0,0.0,0.0,-0.1,0.0,0.0,-0.2,0.0,-0.1,-0.3,-0.1,-0.1,-0.079,-0.741,0.0,0.0,-1.372,-0.12,-0.079,-0.741,-1.492,-2.313,-0.415,-0.773
4,A.J. Greer,A.J..GREER,8478421,21-22,N.J,L,L,1996-12-14,24,2015,2.0,39.0,2,17.05,100.0,40.08,4

In [16]:
eh_df[(eh_df['Position'] != eh_df['Position1'])]

,Player,EH_ID,API ID,Season,Team,Position,Shoots,Birthday,Age,Draft Yr,Draft Rd,Draft Ov,GP,TOI,GF%,SF%,FF%,CF%,xGF%,GF/60,GA/60,SF/60,SA/60,FF/60,FA/60,CF/60,CA/60,xGF/60,xGA/60,G±/60,S±/60,F±/60,C±/60,xG±/60,Sh%,Sv%,Position1,xEVO_GAR/60,xEVD_GAR/60,xPPO_GAR/60,xSHD_GAR/60,xTake_GAR/60,xDraw_GAR/60,xOff_GAR/60,xDef_GAR/60,xPens_GAR/60,xGAR/60,xWAR/60,xSPAR/60,xEVO_GAR,xEVD_GAR,xPPO_GAR,xSHD_GAR,xTake_GAR,xDraw_GAR,xOff_GAR,xDef_GAR,xPens_GAR,xGAR,xWAR,xSPAR,G/60,A1/60,A2/60,Points/60,iSF/60,iFF/60,iCF/60,ixG/60,Sh%1,FSh%,xFSh%,iBLK/60,GIVE/60,TAKE/60,iHF/60,iHA/60,iPENT2/60,iPEND2/60,iPENT5/60,iPEND5/60,iPEN±/60,FOW/60,FOL/60,FO±/60,G,A1,A2,Points,iSF,iFF,iCF,ixG,iBLK,GIVE,TAKE,iHF,iHA,iPENT2,iPEND2,iPENT5,iPEND5,iPEN±,FOW,FOL,FO±,GF,GA,SF,SA,FF,FA,CF,CA,xGF,xGA,G±,S±,F±,C±,xG±,TOI_All,TOI_EV,TOI_PP,TOI_SH,EVO_GAR,EVD_GAR,PPO_GAR,SHD_GAR,Take_GAR,Off_GAR,Def_GAR,GAR,WAR,SPAR,EVO_GAR/60,EVD_GAR/60,PPO_GAR/60,SHD_GAR/60,Take_GAR/60,Draw_GAR/60,Off_GAR/60,Def_GAR/60,Pens_GAR/60,GAR/60,WAR/60,SPAR/60
1561,Brandon Prust,BRANDON.PRUST,8471283,08-09,ARI/CGY,C/R,L,1984-03-16,24,2004,3.0,70.0,36,268.5,27.82,52.62,52.66,53.27,56.97,1.12,2.92,27.45,24.72,38.17,34.31,51.44,45.12,2.58,1.95,-1.79,2.74,3.85,6.32,0.63,4.09,88.2,R/C,0.116,0.134,0.018,-0.026,-0.108,0.1,0.114,0.134,-0.009,0.236,0.043,0.082,0.5,0.6,0.0,0.0,-0.5,0.4,0.5,0.6,0.0,1.1,0.2,0.4,0.22,0.22,0.22,0.67,5.14,7.15,8.27,0.63,4.35,3.12,8.78,2.01,2.23,0.89,5.36,4.69,2.01,1.79,3.58,3.58,-0.22,3.8,3.13,0.67,1,1,1,3,23,32,37,2.81,9,10,4,24,21,9,8,16,16,-1,17,14,3,5.03,13.05,122.85,110.6,170.79,153.55,230.18,201.89,11.53,8.71,-8.02,12.25,17.24,28.29,2.82,268.5,262.7,4.3,0.2,-1.7,0.4,0.0,0.0,-0.5,-1.8,0.4,-1.5,-0.3,-0.5,-0.398,0.081,-0.423,-0.744,-0.108,0.1,-0.398,0.081,-0.009,-0.326,-0.059,-0.113
8134,Mathieu Dandenault,MATHIEU.DANDENAULT,8460540,07-08,MTL,D,R,1976-02-03,31,1994,2.0,49.0,61,679.22,35.22,40.74,40.45,40.05,38.89,1.73,3.18,25.15,36.59,34.56,50.89,45.31,67.84,1.88,2.95,-1.45,-11.43,-16.33,-22.52,-1.07,6.88,91.3,D/R,0.332,-0.198,0.721,-0.428,-0.029,-0.017,0.334,-0.22,-0.046,0.038,0.007,0.013,3.3,-2.0,0.0,-0.4,-0.3,-0.2,3.4,-2.4,-0.5,0.4,0.1,0.1,0.8,0.27,0.18,1.24,6.1,8.75,10.42,0.61,13.04,9.09,7.01,2.3,1.68,1.5,6.18,4.86,1.06,0.35,0.0,0.0,-0.71,0.0,0.09,-0.09,9,3,2,14,69,99,118,6.94,26,19,17,70,55,12,4,0,0,-8,0,1,-1,19.59,36.03,284.74,414.16,391.25,576.11,512.95,767.93,21.25,33.39,-16.44,-129.42,-184.86,-254.98,-12.14,679.2,604.9,3.6,61.3,-2.2,-2.7,0.1,-0.6,-0.3,-2.1,-3.3,-5.9,-1.1,-2.1,-0.218,-0.271,1.494,-0.555,-0.029,-0.017,-0.207,-0.297,-0.046,-0.523,-0.095,-0.182
12769,Tyler Toffoli,TYLER.TOFFOLI,8475726,21-22,CGY/MTL,C/R,R,1992-04-24,29,2010,2.0,47.0,43,730.62,49.72,52.81,52.77,53.58,54.6,3.53,3.57,33.55,29.98,45.81,41.0,62.23,53.91,3.83,3.18,-0.04,3.56,4.81,8.32,0.64,10.53,88.09,R/C,-0.159,-0.008,1.178,0.409,0.119,-0.034,0.057,0.022,0.086,0.155,0.028,0.052,-1.5,-0.1,2.1,0.3,1.5,-0.4,0.6,0.2,1.0,1.9,0.3,0.6,1.07,0.74,0.74,2.55,9.44,12.65,15.85,1.1,11.3,8.44,8.73,1.23,1.72,1.31,4.27,2.05,0.16,0.49,0.0,0.0,0.33,0.74,0.74,0.0,13,9,9,31,115,154,193,13.45,15,21,16,52,25,2,6,0,0,4,9,9,0,43.0,43.48,408.53,365.12,557.83,499.31,757.79,656.48,46.59,38.74,-0.48,43.41,58.52,101.31,7.85,730.6,555.9,107.3,42.4,-1.4,0.2,2.8,0.2,1.5,1.4,0.4,2.9,0.5,1.0,-0.152,0.024,1.584,0.316,0.119,-0.034,0.129,0.045,0.086,0.239,0.043,0.081


There are three total players who have a difference here.  For two the same positions are listed, just in a reversed order.

For the last player, Mathieu Dandenault, has D/R listed.  Looking at his history specifically it is clear that he is primarily a defenceman.

I'm comfortable dropping the Position1 column.

In [17]:
eh_df = eh_df.drop(columns = 'Position1')

In [18]:
(eh_df['Sh%'] != eh_df['Sh%1']).sum()

12358

Here we can see that these are two distinctly different stats.  Looking at the info provided by Evolving Hockey, Sh% is the Shooting Percentage for the TEAM while the player is on the ice whereas Sh%1 is that player's personal shooting percentage.  We will change these names to clarify this.

In [19]:
eh_df.rename({'Sh%': 'oiSh%', 'Sh%1':'Sh%'}, axis = 1, inplace = True)

At this point, the Evolving Hockey data is ready to be used, but the CapFriendly data still needs to be modified in order to merge the data appropriately.

In [20]:
eh_df.head()

,Player,EH_ID,API ID,Season,Team,Position,Shoots,Birthday,Age,Draft Yr,Draft Rd,Draft Ov,GP,TOI,GF%,SF%,FF%,CF%,xGF%,GF/60,GA/60,SF/60,SA/60,FF/60,FA/60,CF/60,CA/60,xGF/60,xGA/60,G±/60,S±/60,F±/60,C±/60,xG±/60,oiSh%,Sv%,xEVO_GAR/60,xEVD_GAR/60,xPPO_GAR/60,xSHD_GAR/60,xTake_GAR/60,xDraw_GAR/60,xOff_GAR/60,xDef_GAR/60,xPens_GAR/60,xGAR/60,xWAR/60,xSPAR/60,xEVO_GAR,xEVD_GAR,xPPO_GAR,xSHD_GAR,xTake_GAR,xDraw_GAR,xOff_GAR,xDef_GAR,xPens_GAR,xGAR,xWAR,xSPAR,G/60,A1/60,A2/60,Points/60,iSF/60,iFF/60,iCF/60,ixG/60,Sh%,FSh%,xFSh%,iBLK/60,GIVE/60,TAKE/60,iHF/60,iHA/60,iPENT2/60,iPEND2/60,iPENT5/60,iPEND5/60,iPEN±/60,FOW/60,FOL/60,FO±/60,G,A1,A2,Points,iSF,iFF,iCF,ixG,iBLK,GIVE,TAKE,iHF,iHA,iPENT2,iPEND2,iPENT5,iPEND5,iPEN±,FOW,FOL,FO±,GF,GA,SF,SA,FF,FA,CF,CA,xGF,xGA,G±,S±,F±,C±,xG±,TOI_All,TOI_EV,TOI_PP,TOI_SH,EVO_GAR,EVD_GAR,PPO_GAR,SHD_GAR,Take_GAR,Off_GAR,Def_GAR,GAR,WAR,SPAR,EVO_GAR/60,EVD_GAR/60,PPO_GAR/60,SHD_GAR/60,Take_GAR/60,Draw_GAR/60,Off_GAR/60,Def_GAR/60,Pens_GAR/60,GAR/60,WAR/60,SPAR/60
0,A.J. Greer,A.J..GREER,8478421,16-17,COL,L,L,1996-12-14,19,2015,2.0,39.0,5,68.28,24.4,51.38,56.0,51.91,64.27,0.89,2.75,30.5,28.87,43.54,34.21,55.68,51.58,2.75,1.53,-1.86,1.63,9.33,4.09,1.22,2.91,90.47,0.547,0.776,-0.82,0.584,0.135,-0.125,0.488,0.772,0.011,1.223,0.23,0.446,0.6,0.8,0.0,0.0,0.2,-0.1,0.5,0.8,0.0,1.4,0.3,0.5,0.0,0.0,0.88,0.88,7.03,8.79,9.67,0.9,0.0,0.0,10.2,2.64,1.76,0.88,3.51,5.27,1.76,1.76,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,1,8,10,11,1.02,3,2,1,4,6,2,2,0,0,0,0,0,0,1.01,3.13,34.71,32.85,49.55,38.93,63.36,58.7,3.13,1.74,-2.12,1.86,10.62,4.66,1.39,68.3,63.8,2.9,1.3,-0.5,0.6,0.0,0.0,0.2,-0.6,0.6,0.1,0.0,0.0,-0.491,0.578,-0.792,0.062,0.135,-0.125,-0.504,0.568,0.011,0.06,0.011,0.022
1,A.J. Greer,A.J..GREER,8478421,17-18,COL,L,L,1996-12-14,20,2015,2.0,39.0,17,125.68,67.51,41.38,39.94,40.27,39.82,1.91,0.92,24.45,34.65,31.58,47.5,41.32,61.3,1.69,2.55,0.99,-10.19,-15.92,-19.98,-0.86,7.83,97.34,-0.351,-0.404,-0.068,0.0,-0.122,0.047,-0.345,-0.404,-0.075,-0.81,-0.158,-0.302,-0.7,-0.8,0.0,0.0,-0.3,0.1,-0.7,-0.8,-0.2,-1.7,-0.3,-0.6,0.0,0.95,0.48,1.43,6.21,7.16,9.55,0.32,0.0,0.0,4.4,2.39,0.95,1.43,14.32,7.64,3.82,2.39,0.48,0.48,-1.43,0.48,0.95,-0.48,0,2,1,3,13,15,20,0.66,5,2,3,30,16,8,5,1,1,-3,1,2,-1,4.01,1.93,51.22,72.57,66.15,99.49,86.56,128.41,3.54,5.35,2.08,-21.35,-33.34,-41.85,-1.81,125.7,122.3,2.8,0.0,0.0,-0.6,0.0,0.0,-0.3,-0.1,-0.6,-0.8,-0.2,-0.3,-0.022,-0.301,-0.12,0.0,-0.122,0.047,-0.024,-0.301,-0.075,-0.392,-0.076,-0.146
2,A.J. Greer,A.J..GREER,8478421,18-19,COL,L,L,1996-12-14,21,2015,2.0,39.0,15,94.02,33.44,49.06,50.64,48.87,47.84,1.31,2.6,28.59,29.68,38.85,37.87,49.32,51.59,2.19,2.39,-1.3,-1.09,0.98,-2.27,-0.2,4.58,91.23,-0.067,0.279,0.0,0.0,-0.317,-0.124,-0.067,0.279,-0.442,-0.23,-0.043,-0.081,-0.1,0.4,0.0,0.0,-0.5,-0.2,-0.1,0.4,-0.7,-0.4,-0.1,-0.1,0.64,0.64,0.0,1.28,5.74,7.02,8.93,0.8,11.11,9.09,11.45,3.19,0.64,2.55,12.76,12.76,4.47,1.28,0.0,0.0,-3.19,1.28,1.91,-0.64,1,1,0,2,9,11,14,1.26,5,1,4,20,20,7,2,0,0,-5,2,3,-1,2.05,4.08,44.8,46.51,60.88,59.34,77.28,80.84,3.43,3.74,-2.03,-1.71,1.54,-3.56,-0.31,94.0,93.8,0.0,0.0,-0.8,-0.2,0.0,0.0,-0.5,-0.8,-0.2,-1.7,-0.3,-0.6,-0.539,-0.116,0.0,0.0,-0.317,-0.124,-0.539,-0.116,-0.442,-1.095,-0.204,-0.384
3,A.J. Greer,A.J..GREER,8478421,20-21,N.J,L,L,1996-12-14,23,2015,2.0,39.0,1,8.55,0.0,7.41,21.06,18.63,15.45,0.0,0.0,6.46,80.7,25.54,95.72,25.26,110.32,1.33,7.3,0.0,-74.25,-70.18,-85.05,-5.96,0.0,100.0,0.206,-0.791,0.0,0.0,-1.372,-0.12,0.206,-0.791,-1.492,-2.078,-0.373,-0.695,0.0,-0.1,0.0,0.0,-0.2,0.0,0.0,-0.1,-0.2,-0.3,-0.1,-0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.02,7.02,0.0,7.02,7.02,-7.02,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0,0,0,0,0,1,1,0,1,1,-1,0,0,0,0.0,0.0,0.92,11.5,3.64,13.64,3.6,15.72,0.19,1.04,0.0,-10.58,-10.0,-12.12,-0.85,8.6,8.6,0.0,0.0,0.0,-0.1,0.0,0.0,-0.2,0.0,-0.1,-0.3,-0.1,-0.1,-0.079,-0.741,0.0,0.0,-1.372,-0.12,-0.079,-0.741,-1.492,-2.313,-0.415,-0.773
4,A.J. Greer,A.J..GREER,8478421,21-22,N.J,L,L,1996-12-14,24,2015,2.0,39.0,2,17.05,100.0,40.08,41.61,49.94,53.37,

### Merging Data

<a id = 'merging'></a>

Unfortunately, there is no common ID or index in order to merge this data cleanly.  The best thing to do will be to merge the data on the player's name AND the season he played in.

This will require formatting each dataframe individually prior to merging.  In previous attempts to merge, there are some data cleaning aspects that are better to take place prior to the merge to ensure that the data is effectively transferred once the data is merged together.

In [21]:
#First we will need to adjust the PLAYER category to match the format in the EH data
#I will split the data on the first white space that would be between the index number and the player's name
money_df[['number', 'Player']] = money_df['PLAYER'].str.split(' ', n=1, expand = True)

In [22]:
money_df.head()

,PLAYER,TEAM,AGE,POS,HANDED,GP,G,A,P,P/GP,+/-,Sh,Sh%,TOI,W,L,SO,GAA,Sv%,CLAUSE,EXPIRY,CAP HIT,SALARY,Year,number,Player
0,1. Jaromir Jagr,-,35,RW,Left,0,0,0,0,0.00,0,0,0.00,NaN,-,-,-,-,-,NaN,UFA,"$7,920,000","$8,360,000",2008,1.,Jaromir Jagr
1,2. Brad Richards,-,27,"C, LW",Left,0,0,0,0,0.00,0,0,0.00,NaN,-,-,-,-,-,NTC,UFA,"$7,800,000","$7,800,000",2008,2.,Brad Richards
2,3. Nicklas Lidstrom,-,37,D,Left,0,0,0,0,0.00,0,0,0.00,NaN,-,-,-,-,-,NMC,UFA,"$7,600,000","$7,600,000",2008,3.,Nicklas Lidstrom
3,4. Zdeno Chara,NYI,30,LD,Left,0,0,0,0,0.00,0,0,0.00,NaN,-,-,-,-,-,NTC,UFA,"$7,500,000","$7,500,000",2008,4.,Zdeno Chara
4,5. Scott Gomez,-,27,C,Left,0,0,0,0,0.00,0,0,0.00,NaN,-,-,-,-,-,NaN,UFA,"$7,357,143","$10,000,000",2008,5.,Scott Gomez


With this modified, we are going to make a new dataframe only with the pertinent financial data that we want to merge with the Evolving Hockey statistics.

In [23]:
cf_df = money_df[['Player', 'AGE', 'Year', 'CAP HIT', 'SALARY', ]]

The year in the Cap Friendly data corresponds with the second year of the Evolving Hockey data (2008 = 07-08).  We will need to modify this to ensure we can merge the player/years appropriately.  We'll loop through the EH dataframe to convert the season into the year.

In [24]:
eh_df['Year'] = eh_df['Season']
for i in range(0, eh_df.shape[0]):
    eh_df['Year'][i] = int('20'+eh_df['Year'][i][-2:])

In [25]:
eh_df.head()

,Player,EH_ID,API ID,Season,Team,Position,Shoots,Birthday,Age,Draft Yr,Draft Rd,Draft Ov,GP,TOI,GF%,SF%,FF%,CF%,xGF%,GF/60,GA/60,SF/60,SA/60,FF/60,FA/60,CF/60,CA/60,xGF/60,xGA/60,G±/60,S±/60,F±/60,C±/60,xG±/60,oiSh%,Sv%,xEVO_GAR/60,xEVD_GAR/60,xPPO_GAR/60,xSHD_GAR/60,xTake_GAR/60,xDraw_GAR/60,xOff_GAR/60,xDef_GAR/60,xPens_GAR/60,xGAR/60,xWAR/60,xSPAR/60,xEVO_GAR,xEVD_GAR,xPPO_GAR,xSHD_GAR,xTake_GAR,xDraw_GAR,xOff_GAR,xDef_GAR,xPens_GAR,xGAR,xWAR,xSPAR,G/60,A1/60,A2/60,Points/60,iSF/60,iFF/60,iCF/60,ixG/60,Sh%,FSh%,xFSh%,iBLK/60,GIVE/60,TAKE/60,iHF/60,iHA/60,iPENT2/60,iPEND2/60,iPENT5/60,iPEND5/60,iPEN±/60,FOW/60,FOL/60,FO±/60,G,A1,A2,Points,iSF,iFF,iCF,ixG,iBLK,GIVE,TAKE,iHF,iHA,iPENT2,iPEND2,iPENT5,iPEND5,iPEN±,FOW,FOL,FO±,GF,GA,SF,SA,FF,FA,CF,CA,xGF,xGA,G±,S±,F±,C±,xG±,TOI_All,TOI_EV,TOI_PP,TOI_SH,EVO_GAR,EVD_GAR,PPO_GAR,SHD_GAR,Take_GAR,Off_GAR,Def_GAR,GAR,WAR,SPAR,EVO_GAR/60,EVD_GAR/60,PPO_GAR/60,SHD_GAR/60,Take_GAR/60,Draw_GAR/60,Off_GAR/60,Def_GAR/60,Pens_GAR/60,GAR/60,WAR/60,SPAR/60,Year
0,A.J. Greer,A.J..GREER,8478421,16-17,COL,L,L,1996-12-14,19,2015,2.0,39.0,5,68.28,24.4,51.38,56.0,51.91,64.27,0.89,2.75,30.5,28.87,43.54,34.21,55.68,51.58,2.75,1.53,-1.86,1.63,9.33,4.09,1.22,2.91,90.47,0.547,0.776,-0.82,0.584,0.135,-0.125,0.488,0.772,0.011,1.223,0.23,0.446,0.6,0.8,0.0,0.0,0.2,-0.1,0.5,0.8,0.0,1.4,0.3,0.5,0.0,0.0,0.88,0.88,7.03,8.79,9.67,0.9,0.0,0.0,10.2,2.64,1.76,0.88,3.51,5.27,1.76,1.76,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,1,8,10,11,1.02,3,2,1,4,6,2,2,0,0,0,0,0,0,1.01,3.13,34.71,32.85,49.55,38.93,63.36,58.7,3.13,1.74,-2.12,1.86,10.62,4.66,1.39,68.3,63.8,2.9,1.3,-0.5,0.6,0.0,0.0,0.2,-0.6,0.6,0.1,0.0,0.0,-0.491,0.578,-0.792,0.062,0.135,-0.125,-0.504,0.568,0.011,0.06,0.011,0.022,2017
1,A.J. Greer,A.J..GREER,8478421,17-18,COL,L,L,1996-12-14,20,2015,2.0,39.0,17,125.68,67.51,41.38,39.94,40.27,39.82,1.91,0.92,24.45,34.65,31.58,47.5,41.32,61.3,1.69,2.55,0.99,-10.19,-15.92,-19.98,-0.86,7.83,97.34,-0.351,-0.404,-0.068,0.0,-0.122,0.047,-0.345,-0.404,-0.075,-0.81,-0.158,-0.302,-0.7,-0.8,0.0,0.0,-0.3,0.1,-0.7,-0.8,-0.2,-1.7,-0.3,-0.6,0.0,0.95,0.48,1.43,6.21,7.16,9.55,0.32,0.0,0.0,4.4,2.39,0.95,1.43,14.32,7.64,3.82,2.39,0.48,0.48,-1.43,0.48,0.95,-0.48,0,2,1,3,13,15,20,0.66,5,2,3,30,16,8,5,1,1,-3,1,2,-1,4.01,1.93,51.22,72.57,66.15,99.49,86.56,128.41,3.54,5.35,2.08,-21.35,-33.34,-41.85,-1.81,125.7,122.3,2.8,0.0,0.0,-0.6,0.0,0.0,-0.3,-0.1,-0.6,-0.8,-0.2,-0.3,-0.022,-0.301,-0.12,0.0,-0.122,0.047,-0.024,-0.301,-0.075,-0.392,-0.076,-0.146,2018
2,A.J. Greer,A.J..GREER,8478421,18-19,COL,L,L,1996-12-14,21,2015,2.0,39.0,15,94.02,33.44,49.06,50.64,48.87,47.84,1.31,2.6,28.59,29.68,38.85,37.87,49.32,51.59,2.19,2.39,-1.3,-1.09,0.98,-2.27,-0.2,4.58,91.23,-0.067,0.279,0.0,0.0,-0.317,-0.124,-0.067,0.279,-0.442,-0.23,-0.043,-0.081,-0.1,0.4,0.0,0.0,-0.5,-0.2,-0.1,0.4,-0.7,-0.4,-0.1,-0.1,0.64,0.64,0.0,1.28,5.74,7.02,8.93,0.8,11.11,9.09,11.45,3.19,0.64,2.55,12.76,12.76,4.47,1.28,0.0,0.0,-3.19,1.28,1.91,-0.64,1,1,0,2,9,11,14,1.26,5,1,4,20,20,7,2,0,0,-5,2,3,-1,2.05,4.08,44.8,46.51,60.88,59.34,77.28,80.84,3.43,3.74,-2.03,-1.71,1.54,-3.56,-0.31,94.0,93.8,0.0,0.0,-0.8,-0.2,0.0,0.0,-0.5,-0.8,-0.2,-1.7,-0.3,-0.6,-0.539,-0.116,0.0,0.0,-0.317,-0.124,-0.539,-0.116,-0.442,-1.095,-0.204,-0.384,2019
3,A.J. Greer,A.J..GREER,8478421,20-21,N.J,L,L,1996-12-14,23,2015,2.0,39.0,1,8.55,0.0,7.41,21.06,18.63,15.45,0.0,0.0,6.46,80.7,25.54,95.72,25.26,110.32,1.33,7.3,0.0,-74.25,-70.18,-85.05,-5.96,0.0,100.0,0.206,-0.791,0.0,0.0,-1.372,-0.12,0.206,-0.791,-1.492,-2.078,-0.373,-0.695,0.0,-0.1,0.0,0.0,-0.2,0.0,0.0,-0.1,-0.2,-0.3,-0.1,-0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.02,7.02,0.0,7.02,7.02,-7.02,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0,0,0,0,0,1,1,0,1,1,-1,0,0,0,0.0,0.0,0.92,11.5,3.64,13.64,3.6,15.72,0.19,1.04,0.0,-10.58,-10.0,-12.12,-0.85,8.6,8.6,0.0,0.0,0.0,-0.1,0.0,0.0,-0.2,0.0,-0.1,-0.3,-0.1,-0.1,-0.079,-0.741,0.0,0.0,-1.372,-0.12,-0.079,-0.741,-1.492,-2.313,-0.415,-0.773,2021
4,A.J. Greer,A.J..GREER,8478421,21-22,N.J,L,L,1996-12-14,24,2015,2.0,39.0,2,17.05,100.0

In [26]:
cf_df.head()

,Player,AGE,Year,CAP HIT,SALARY
0,Jaromir Jagr,35,2008,"$7,920,000","$8,360,000"
1,Brad Richards,27,2008,"$7,800,000","$7,800,000"
2,Nicklas Lidstrom,37,2008,"$7,600,000","$7,600,000"
3,Zdeno Chara,30,2008,"$7,500,000","$7,500,000"
4,Scott Gomez,27,2008,"$7,357,143","$10,000,000"


In [27]:
cf_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19439 entries, 0 to 19438
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Player   19439 non-null  object
 1   AGE      19439 non-null  object
 2   Year     19439 non-null  int64 
 3   CAP HIT  19439 non-null  object
 4   SALARY   19439 non-null  object
dtypes: int64(1), object(4)
memory usage: 759.5+ KB


With the player names and years properly modified, I will now transform the `CAP HIT` and `SALARY` columns so that they can be used as integers.

In [28]:
#Removing the punctuation with nothing.  $7,500,000 to 7500000
cf_df['Cap_Hit'] = cf_df['CAP HIT'].replace({'\$': '', ',': ''}, regex=True)

/var/folders/dm/5cg2631s7057bxm5mx52z5fr0000gn/T/ipykernel_71194/1807641103.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cf_df['Cap_Hit'] = cf_df['CAP HIT'].replace({'\$': '', ',': ''}, regex=True)


In [29]:
cf_df['Cap_Hit'] = pd.to_numeric(cf_df['Cap_Hit'])

/var/folders/dm/5cg2631s7057bxm5mx52z5fr0000gn/T/ipykernel_71194/1469276086.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cf_df['Cap_Hit'] = pd.to_numeric(cf_df['Cap_Hit'])


In [30]:
#Putting the units into millions.  7500000 into 7.5
cf_df['Cap_Hit'] = round(cf_df['Cap_Hit']/1000000, 3)

/var/folders/dm/5cg2631s7057bxm5mx52z5fr0000gn/T/ipykernel_71194/2520254114.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cf_df['Cap_Hit'] = round(cf_df['Cap_Hit']/1000000, 3)


In [31]:
#Repeating the process with SALARY
cf_df['Salary'] = cf_df['SALARY'].replace({'\$': '', ',': ''}, regex=True)

/var/folders/dm/5cg2631s7057bxm5mx52z5fr0000gn/T/ipykernel_71194/2094253197.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cf_df['Salary'] = cf_df['SALARY'].replace({'\$': '', ',': ''}, regex=True)


In [32]:
cf_df['Salary'] = pd.to_numeric(cf_df['Salary'])

/var/folders/dm/5cg2631s7057bxm5mx52z5fr0000gn/T/ipykernel_71194/2913349225.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cf_df['Salary'] = pd.to_numeric(cf_df['Salary'])


In [33]:
cf_df['Salary'] = round(cf_df['Salary']/1000000, 3)

/var/folders/dm/5cg2631s7057bxm5mx52z5fr0000gn/T/ipykernel_71194/1673885138.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cf_df['Salary'] = round(cf_df['Salary']/1000000, 3)


In [34]:
cf_df.head()

,Player,AGE,Year,CAP HIT,SALARY,Cap_Hit,Salary
0,Jaromir Jagr,35,2008,"$7,920,000","$8,360,000",7.920,8.36
1,Brad Richards,27,2008,"$7,800,000","$7,800,000",7.800,7.80
2,Nicklas Lidstrom,37,2008,"$7,600,000","$7,600,000",7.600,7.60
3,Zdeno Chara,30,2008,"$7,500,000","$7,500,000",7.500,7.50
4,Scott Gomez,27,2008,"$7,357,143","$10,000,000",7.357,10.00


In [35]:
#We are going to minimize the cf data further to elimiate unnecssary transfer of data and only merge the financial
#info
cf_df = cf_df[['Player', 'Year', 'AGE', 'Cap_Hit', 'Salary']]

In [36]:
cf_df.head()

,Player,Year,AGE,Cap_Hit,Salary
0,Jaromir Jagr,2008,35,7.920,8.36
1,Brad Richards,2008,27,7.800,7.80
2,Nicklas Lidstrom,2008,37,7.600,7.60
3,Zdeno Chara,2008,30,7.500,7.50
4,Scott Gomez,2008,27,7.357,10.00


In [37]:
#In preparation for merging the data we are going to make sure there is no punctuation and everything is in lower case
#To minimize errors in merging on the Player's name
eh_df['Player'] = eh_df['Player'].str.lower()
cf_df['Player'] = cf_df['Player'].str.lower()

In [38]:
import string

In [39]:
#Quick function to remove punctuation since the column is titled 'Player' on both dataframes
def remove_punc(df):
    for name in range(0, df.shape[0]):
        df['Player'][name] = df['Player'][name].translate(str.maketrans('', '', string.punctuation))

In [40]:
remove_punc(eh_df)

In [41]:
remove_punc(cf_df)

/var/folders/dm/5cg2631s7057bxm5mx52z5fr0000gn/T/ipykernel_71194/3033421912.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Player'][name] = df['Player'][name].translate(str.maketrans('', '', string.punctuation))


In [42]:
eh_df.head()

,Player,EH_ID,API ID,Season,Team,Position,Shoots,Birthday,Age,Draft Yr,Draft Rd,Draft Ov,GP,TOI,GF%,SF%,FF%,CF%,xGF%,GF/60,GA/60,SF/60,SA/60,FF/60,FA/60,CF/60,CA/60,xGF/60,xGA/60,G±/60,S±/60,F±/60,C±/60,xG±/60,oiSh%,Sv%,xEVO_GAR/60,xEVD_GAR/60,xPPO_GAR/60,xSHD_GAR/60,xTake_GAR/60,xDraw_GAR/60,xOff_GAR/60,xDef_GAR/60,xPens_GAR/60,xGAR/60,xWAR/60,xSPAR/60,xEVO_GAR,xEVD_GAR,xPPO_GAR,xSHD_GAR,xTake_GAR,xDraw_GAR,xOff_GAR,xDef_GAR,xPens_GAR,xGAR,xWAR,xSPAR,G/60,A1/60,A2/60,Points/60,iSF/60,iFF/60,iCF/60,ixG/60,Sh%,FSh%,xFSh%,iBLK/60,GIVE/60,TAKE/60,iHF/60,iHA/60,iPENT2/60,iPEND2/60,iPENT5/60,iPEND5/60,iPEN±/60,FOW/60,FOL/60,FO±/60,G,A1,A2,Points,iSF,iFF,iCF,ixG,iBLK,GIVE,TAKE,iHF,iHA,iPENT2,iPEND2,iPENT5,iPEND5,iPEN±,FOW,FOL,FO±,GF,GA,SF,SA,FF,FA,CF,CA,xGF,xGA,G±,S±,F±,C±,xG±,TOI_All,TOI_EV,TOI_PP,TOI_SH,EVO_GAR,EVD_GAR,PPO_GAR,SHD_GAR,Take_GAR,Off_GAR,Def_GAR,GAR,WAR,SPAR,EVO_GAR/60,EVD_GAR/60,PPO_GAR/60,SHD_GAR/60,Take_GAR/60,Draw_GAR/60,Off_GAR/60,Def_GAR/60,Pens_GAR/60,GAR/60,WAR/60,SPAR/60,Year
0,aj greer,A.J..GREER,8478421,16-17,COL,L,L,1996-12-14,19,2015,2.0,39.0,5,68.28,24.4,51.38,56.0,51.91,64.27,0.89,2.75,30.5,28.87,43.54,34.21,55.68,51.58,2.75,1.53,-1.86,1.63,9.33,4.09,1.22,2.91,90.47,0.547,0.776,-0.82,0.584,0.135,-0.125,0.488,0.772,0.011,1.223,0.23,0.446,0.6,0.8,0.0,0.0,0.2,-0.1,0.5,0.8,0.0,1.4,0.3,0.5,0.0,0.0,0.88,0.88,7.03,8.79,9.67,0.9,0.0,0.0,10.2,2.64,1.76,0.88,3.51,5.27,1.76,1.76,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,1,8,10,11,1.02,3,2,1,4,6,2,2,0,0,0,0,0,0,1.01,3.13,34.71,32.85,49.55,38.93,63.36,58.7,3.13,1.74,-2.12,1.86,10.62,4.66,1.39,68.3,63.8,2.9,1.3,-0.5,0.6,0.0,0.0,0.2,-0.6,0.6,0.1,0.0,0.0,-0.491,0.578,-0.792,0.062,0.135,-0.125,-0.504,0.568,0.011,0.06,0.011,0.022,2017
1,aj greer,A.J..GREER,8478421,17-18,COL,L,L,1996-12-14,20,2015,2.0,39.0,17,125.68,67.51,41.38,39.94,40.27,39.82,1.91,0.92,24.45,34.65,31.58,47.5,41.32,61.3,1.69,2.55,0.99,-10.19,-15.92,-19.98,-0.86,7.83,97.34,-0.351,-0.404,-0.068,0.0,-0.122,0.047,-0.345,-0.404,-0.075,-0.81,-0.158,-0.302,-0.7,-0.8,0.0,0.0,-0.3,0.1,-0.7,-0.8,-0.2,-1.7,-0.3,-0.6,0.0,0.95,0.48,1.43,6.21,7.16,9.55,0.32,0.0,0.0,4.4,2.39,0.95,1.43,14.32,7.64,3.82,2.39,0.48,0.48,-1.43,0.48,0.95,-0.48,0,2,1,3,13,15,20,0.66,5,2,3,30,16,8,5,1,1,-3,1,2,-1,4.01,1.93,51.22,72.57,66.15,99.49,86.56,128.41,3.54,5.35,2.08,-21.35,-33.34,-41.85,-1.81,125.7,122.3,2.8,0.0,0.0,-0.6,0.0,0.0,-0.3,-0.1,-0.6,-0.8,-0.2,-0.3,-0.022,-0.301,-0.12,0.0,-0.122,0.047,-0.024,-0.301,-0.075,-0.392,-0.076,-0.146,2018
2,aj greer,A.J..GREER,8478421,18-19,COL,L,L,1996-12-14,21,2015,2.0,39.0,15,94.02,33.44,49.06,50.64,48.87,47.84,1.31,2.6,28.59,29.68,38.85,37.87,49.32,51.59,2.19,2.39,-1.3,-1.09,0.98,-2.27,-0.2,4.58,91.23,-0.067,0.279,0.0,0.0,-0.317,-0.124,-0.067,0.279,-0.442,-0.23,-0.043,-0.081,-0.1,0.4,0.0,0.0,-0.5,-0.2,-0.1,0.4,-0.7,-0.4,-0.1,-0.1,0.64,0.64,0.0,1.28,5.74,7.02,8.93,0.8,11.11,9.09,11.45,3.19,0.64,2.55,12.76,12.76,4.47,1.28,0.0,0.0,-3.19,1.28,1.91,-0.64,1,1,0,2,9,11,14,1.26,5,1,4,20,20,7,2,0,0,-5,2,3,-1,2.05,4.08,44.8,46.51,60.88,59.34,77.28,80.84,3.43,3.74,-2.03,-1.71,1.54,-3.56,-0.31,94.0,93.8,0.0,0.0,-0.8,-0.2,0.0,0.0,-0.5,-0.8,-0.2,-1.7,-0.3,-0.6,-0.539,-0.116,0.0,0.0,-0.317,-0.124,-0.539,-0.116,-0.442,-1.095,-0.204,-0.384,2019
3,aj greer,A.J..GREER,8478421,20-21,N.J,L,L,1996-12-14,23,2015,2.0,39.0,1,8.55,0.0,7.41,21.06,18.63,15.45,0.0,0.0,6.46,80.7,25.54,95.72,25.26,110.32,1.33,7.3,0.0,-74.25,-70.18,-85.05,-5.96,0.0,100.0,0.206,-0.791,0.0,0.0,-1.372,-0.12,0.206,-0.791,-1.492,-2.078,-0.373,-0.695,0.0,-0.1,0.0,0.0,-0.2,0.0,0.0,-0.1,-0.2,-0.3,-0.1,-0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.02,7.02,0.0,7.02,7.02,-7.02,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0,0,0,0,0,1,1,0,1,1,-1,0,0,0,0.0,0.0,0.92,11.5,3.64,13.64,3.6,15.72,0.19,1.04,0.0,-10.58,-10.0,-12.12,-0.85,8.6,8.6,0.0,0.0,0.0,-0.1,0.0,0.0,-0.2,0.0,-0.1,-0.3,-0.1,-0.1,-0.079,-0.741,0.0,0.0,-1.372,-0.12,-0.079,-0.741,-1.492,-2.313,-0.415,-0.773,2021
4,aj greer,A.J..GREER,8478421,21-22,N.J,L,L,1996-12-14,24,2015,2.0,39.0,2,17.05,100.0,40.08,41.

On a prior merge attempt, I found that former players that have passed away had their ages marked as 'Deceased (age)'.  I will rectify this next.

In [43]:
cf_df['AGE'] = cf_df['AGE'].astype('str')

In [44]:
cf_df.loc[cf_df['AGE'].str.match(r'Deceased') == True] = cf_df['AGE'][i][10:12]
#Counting through the number of characters it takes to type Deceased, I have grabbed the two immediately after
#which represents the players true age

In [45]:
cf_df.reset_index(inplace = True)
cf_df.drop(columns=['index'], inplace = True)

In [46]:
cf_df['AGE'].isna().sum()

0

On prior attempts to merge this data, I encountered a few problems that would result in approximately 2% of the data having null values under `Cap_Hit` and `Salary`.  These null values fell under 4 categories:

#1: Mismatched names from one dataset to the other.  There are not too many but they have to be hard-coded, and that will be done here prior to merging the data.

#2: Players with the same names.  Evolving Hockey had distinctive names for players with the same names, CapFriendly does not.  We have to make these changes now to the CapFriendly database to ensure the merge is effective.  This also requires hard-coding.

#3: Players that CapFriendly had record of, but somehow was not recorded in the scraping.  They will be hard-coded after the merge

#4: Players (mostly from 10+ years ago) that CapFriendly had no record at all for their salaries.  These players will be dropped from the dataset after the merge

In [47]:
cf_df.loc[(cf_df['Player']) == 'erik gustafsson', 'Player'] = 'erik gustafsson d'
cf_df.loc[(cf_df['Player']) == 'anthony deangelo', 'Player'] = 'tony deangelo'
cf_df.loc[(cf_df['Player']) == 'martin st  louis', 'Player'] = 'martin st louis'
cf_df.loc[(cf_df['Player']) == 'cristoval nieves', 'Player'] = 'boo nieves'
cf_df.loc[(cf_df['Player']) == 'mikko lehtonen', 'Player'] = 'mikko lehtonen d'
cf_df.loc[(cf_df['Player']) == 'yevgeni medvedev', 'Player'] = 'evgeny medvedev'
cf_df.loc[((cf_df['Player']) == 'colin white') & (cf_df['Year'] == 2017), 'Player'] = 'colin white c'
cf_df.loc[((cf_df['Player']) == 'colin white') & (cf_df['Year'] == 2018), 'Player'] = 'colin white c'
cf_df.loc[((cf_df['Player']) == 'colin white') & (cf_df['Year'] == 2019), 'Player'] = 'colin white c'
cf_df.loc[((cf_df['Player']) == 'colin white') & (cf_df['Year'] == 2020), 'Player'] = 'colin white c'
cf_df.loc[((cf_df['Player']) == 'colin white') & (cf_df['Year'] == 2021), 'Player'] = 'colin white c'
cf_df.loc[((cf_df['Player']) == 'colin white') & (cf_df['Year'] == 2022), 'Player'] = 'colin white c'

eh_df.loc[(eh_df['Player']) == 'martin  st louis', 'Player'] = 'martin st louis'

cf_df.loc[((cf_df['Player']) == 'erik karlsson') & (cf_df['Year'] == 2015) & (cf_df['AGE'] == str(19)), 
              ['Player']] = ['erik karlsson2']
cf_df.loc[((cf_df['Player']) == 'erik karlsson') & (cf_df['Year'] == 2016) & (cf_df['AGE'] == str(20)), 
              ['Player']] = ['erik karlsson2']
cf_df.loc[((cf_df['Player']) == 'erik karlsson') & (cf_df['Year'] == 2017) & (cf_df['AGE'] == str(21)), 
              ['Player']] = ['erik karlsson2']


cf_df.loc[((cf_df['Player']) == 'sebastian aho') & (cf_df['Year'] == 2018) & (cf_df['AGE'] == str(21)), 
              ['Player']] = ['sebastian aho2']
cf_df.loc[((cf_df['Player']) == 'sebastian aho') & (cf_df['Year'] == 2019) & (cf_df['AGE'] == str(22)), 
              ['Player']] = ['sebastian aho2']
cf_df.loc[((cf_df['Player']) == 'sebastian aho') & (cf_df['Year'] == 2020) & (cf_df['AGE'] == str(23)), 
              ['Player']] = ['sebastian aho2']
cf_df.loc[((cf_df['Player']) == 'sebastian aho') & (cf_df['Year'] == 2021) & (cf_df['AGE'] == str(24)), 
              ['Player']] = ['sebastian aho2']
cf_df.loc[((cf_df['Player']) == 'sebastian aho') & (cf_df['Year'] == 2022) & (cf_df['AGE'] == str(25)), 
              ['Player']] = ['sebastian aho2']

In [48]:
cf_df[cf_df['Player'] == 'sebastian aho2']

,Player,Year,AGE,Cap_Hit,Salary
12626,sebastian aho2,2018,21,0.77,0.742
14239,sebastian aho2,2019,22,0.77,0.742
15872,sebastian aho2,2020,23,0.77,0.825
17632,sebastian aho2,2021,24,0.725,0.7
19417,sebastian aho2,2022,25,0.725,0.75


There is some issue merging players based on their full names.  For one, some players had different first names on the different sources (PA Parenteau vs Pierre-Alexandre Parenteau).  Additionally, I couldn't just use a first initial because some players had names that were too similar (Jordie and Jamie Benn).

Therefore, I will use the first 3 letters of the first name to match, and then hard-code the salaries for the players missing values there (such as PA Parenteau).

In [49]:
#In order to get a better chance at matches, we're going to split the Player names into First and Last names
#In hope of matching the player/years together better
cf_df[['First_Name', 'Last_Name']] = cf_df['Player'].str.split(' ', n=1, expand = True)
eh_df[['First_Name', 'Last_Name']] = eh_df['Player'].str.split(' ', n=1, expand = True)

In [50]:
#Then we'll reduce the First Name to just the first 3 letters in order to still match to an appreviated first name
#In a previous EDA, I caught that I had 2 x J Benn (Jamie and Jordie) and 2 Ri. Nash (Riley and Rick)
cf_df['First_Name'] = cf_df['First_Name'].str[0:3]
eh_df['First_Name'] = eh_df['First_Name'].str[0:3]

In [51]:
cf_df['First_Name']

0        jar
1        bra
2        nic
3        zde
4        sco
        ... 
19434    mic
19435    bri
19436    log
19437    jak
19438    vit
Name: First_Name, Length: 19439, dtype: object

In [52]:
#And now we'll try to match first name (first initial) and last name and Year together.
player_df = eh_df.merge(cf_df, left_on=['First_Name', 'Last_Name', 'Year'], 
                        right_on=['First_Name', 'Last_Name', 'Year'],
                        how='left',
                       suffixes=('', '1'))

In [ ]:
player_df.head()

### Data Cleaning

<a id = 'cleaning'></a>

With the data merged, the focus will now be checking to ensure the data was merged effectively and the remainder of the data cleaning so that the dataset is ready for analysis.

In [53]:
#It looks like we added to the size of the data with the merge.  Potential for duplicate rows.
player_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13530 entries, 0 to 13529
Columns: 153 entries, Player to Salary
dtypes: object(153)
memory usage: 15.9+ MB


In [54]:
#Quick for loop to turn appropriate columns to numeric
for col in player_df.columns:
    try:
        player_df[col] = pd.to_numeric(player_df[col])
    except:
        pass

In [55]:
player_df['Birthday'] = pd.to_datetime(player_df['Birthday'])

In [56]:
player_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13530 entries, 0 to 13529
Columns: 153 entries, Player to Salary
dtypes: datetime64[ns](1), float64(118), int64(25), object(9)
memory usage: 15.9+ MB


In [57]:
player_df.duplicated().sum()

154

In [58]:
player_df[player_df.duplicated()]

,Player,EH_ID,API ID,Season,Team,Position,Shoots,Birthday,Age,Draft Yr,Draft Rd,Draft Ov,GP,TOI,GF%,SF%,FF%,CF%,xGF%,GF/60,GA/60,SF/60,SA/60,FF/60,FA/60,CF/60,CA/60,xGF/60,xGA/60,G±/60,S±/60,F±/60,C±/60,xG±/60,oiSh%,Sv%,xEVO_GAR/60,xEVD_GAR/60,xPPO_GAR/60,xSHD_GAR/60,xTake_GAR/60,xDraw_GAR/60,xOff_GAR/60,xDef_GAR/60,xPens_GAR/60,xGAR/60,xWAR/60,xSPAR/60,xEVO_GAR,xEVD_GAR,xPPO_GAR,xSHD_GAR,xTake_GAR,xDraw_GAR,xOff_GAR,xDef_GAR,xPens_GAR,xGAR,xWAR,xSPAR,G/60,A1/60,A2/60,Points/60,iSF/60,iFF/60,iCF/60,ixG/60,Sh%,FSh%,xFSh%,iBLK/60,GIVE/60,TAKE/60,iHF/60,iHA/60,iPENT2/60,iPEND2/60,iPENT5/60,iPEND5/60,iPEN±/60,FOW/60,FOL/60,FO±/60,G,A1,A2,Points,iSF,iFF,iCF,ixG,iBLK,GIVE,TAKE,iHF,iHA,iPENT2,iPEND2,iPENT5,iPEND5,iPEN±,FOW,FOL,FO±,GF,GA,SF,SA,FF,FA,CF,CA,xGF,xGA,G±,S±,F±,C±,xG±,TOI_All,TOI_EV,TOI_PP,TOI_SH,EVO_GAR,EVD_GAR,PPO_GAR,SHD_GAR,Take_GAR,Off_GAR,Def_GAR,GAR,WAR,SPAR,EVO_GAR/60,EVD_GAR/60,PPO_GAR/60,SHD_GAR/60,Take_GAR/60,Draw_GAR/60,Off_GAR/60,Def_GAR/60,Pens_GAR/60,GAR/60,WAR/60,SPAR/60,Year,First_Name,Last_Name,Player1,AGE,Cap_Hit,Salary
164,adam pardy,ADAM.PARDY,8471385,13-14,WPG,D,L,1984-03-29,29,2004,6.0,173.0,60,865.25,49.79,51.02,51.04,50.28,49.92,2.06,2.08,28.23,27.11,39.26,37.66,52.16,51.59,2.37,2.38,-0.02,1.13,1.59,0.58,-0.01,7.31,92.33,0.039,0.350,0.128,-0.090,-0.031,-0.013,0.040,0.319,-0.044,0.310,0.059,0.110,0.5,4.7,0.0,-0.1,-0.4,-0.2,0.5,4.6,-0.6,4.5,0.8,1.6,0.00,0.14,0.28,0.42,3.26,4.72,6.93,0.19,0.00,0.00,3.99,3.81,2.15,1.04,5.34,8.11,1.04,0.42,0.14,0.14,-0.62,0.00,0.00,0.00,0,2,4,6,47,68,100,2.71,55,31,15,77,117,15,6,2,2,-9,0,0,0,29.75,30.00,407.17,390.94,566.14,543.16,752.25,743.94,34.14,34.25,-0.25,16.23,22.98,8.31,-0.11,865.2,799.5,1.5,59.7,1.3,4.0,0.0,-0.5,-0.4,1.3,3.5,4.2,0.8,1.5,0.100,0.299,-0.074,-0.496,-0.031,-0.013,0.099,0.244,-0.044,0.290,0.055,0.103,2014,ada,pardy,adam pardy,29.0,0.600,0.600
168,adam pardy,ADAM.PARDY,8471385,16-17,NSH,D,L,1984-03-29,32,2004,6.0,173.0,4,43.97,33.55,47.62,43.81,41.92,37.54,1.38,2.73,31.75,34.93,39.23,50.31,48.14,66.71,1.83,3.04,-1.35,-3.18,-11.08,-18.57,-1.21,4.34,92.19,-0.042,-0.518,0.000,0.000,-0.611,-0.061,-0.042,-0.518,-0.672,-1.227,-0.231,-0.447,0.0,-0.4,0.0,0.0,-0.4,0.0,0.0,-0.4,-0.5,-0.9,-0.2,-0.3,0.00,0.00,0.00,0.00,5.46,6.82,8.19,0.14,0.00,0.00,2.00,4.09,0.00,1.36,4.09,8.19,4.09,0.00,0.00,0.00,-4.09,0.00,0.00,0.00,0,0,0,0,4,5,6,0.10,3,0,1,3,6,3,0,0,0,-3,0,0,0,1.01,2.00,23.27,25.60,28.75,36.87,35.28,48.89,1.34,2.23,-0.99,-2.33,-8.12,-13.61,-0.89,44.0,43.5,0.0,0.3,-0.3,-0.4,0.0,0.0,-0.4,-0.3,-0.4,-1.2,-0.2,-0.4,-0.402,-0.556,0.000,0.173,-0.611,-0.061,-0.402,-0.550,-0.672,-1.619,-0.305,-0.590,2017,ada,pardy,adam pardy,32.0,0.575,0.575
429,alex picard,ALEX.PICARD,8470678,11-12,PIT,D,L,1985-07-05,26,2003,3.0,85.0,17,223.42,59.34,58.90,59.65,58.95,55.49,3.50,2.40,31.24,21.79,43.80,29.63,57.96,40.35,2.48,1.99,1.10,9.44,14.17,17.61,0.49,11.19,89.01,0.036,0.299,-0.502,0.230,0.035,0.027,0.017,0.297,0.062,0.363,0.063,0.118,0.1,1.0,-0.1,0.0,0.1,0.1,0.1,1.1,0.2,1.4,0.2,0.4,0.00,0.54,0.54,1.07,2.69,3.49,5.10,0.15,0.00,0.00,4.31,3.76,0.81,1.07,3.22,6.71,0.54,0.54,0.00,0.00,0.00,0.00,0.00,0.00,0,2,2,4,10,13,19,0.56,14,3,4,12,25,2,2,0,0,0,0,0,0,13.02,8.92,116.31,81.15,163.10,110.34,215.82,150.26,9.25,7.42,4.10,35.16,52.76,65.56,1.83,223.4,210.1,7.6,3.9,2.9,0.3,-0.1,0.0,0.1,2.9,0.3,3.4,0.6,1.1,0.839,0.085,-0.424,0.154,0.035,0.027,0.795,0.086,0.062,0.919,0.160,0.297,2012,ale,picard,alexandre picard,25.0,0.600,0.600
782,andrew ladd,ANDREW.LADD,8471217,21-22,ARI,L,L,1985-12-12,35,2004,1.0,4.0,42,584.38,43.20,44.59,45.94,46.46,48.80,2.21,2.90,24.67,30.66,35.63,41.92,48.00,55.30,2.27,2.38,-0.70,-5.98,-6.29,-7.30,-0.11,8.96,90.53,0.099,0.039,-0.340,1.579,-0.079,-0.061,0.045,0.102,-0.140,-0.009,-0.002,-0.003,0.8,0.3,-0.4,0.6,-0.8,-0.6,0.4,0.9,-1.4,-0.1,0.0,0.0,0.62,0.21,0.10,0.92,4.83,6.98,9.45,0.62,12.77,8.82,8.82,2.77,1.85,1.54,6.37,4.52,1.33,0.41,0.10,0.10,-0.92,0.51,0.21,0.31,6,2,1,9,47,68,92,6.00,27,18,15,62,44,13,4,1,1,-9,5,2,3,21.52,28.29,240.30,298.59,346.99,408.30,467.48,5

It is unclear why these rows are duplicated.  However, it is a small sample of the data and we are still keeping one of the copies of the data, so I'm fine with dropping the duplicate rows.

In [59]:
player_df.drop_duplicates(inplace = True)

Thinking about how I want to include data into the analysis, I am going to set some boundries on the data we're going to look at.

Using the NHL definition of a rookie, a player can only be a rookie of they have played less than 25 games the previous year.  This seems a fair cut off point to drop players who were either not good enough to be regularly in the NHL or had their season seriously affected by injury.  Ideally, we are maximizing the number of players who were able to play the majority of the season and have a good sample of their ability over the course of that season of play.

We are also only going to look at the last 10 years worth of data.  With changes in salary structure and less reliable salary data the further we go into the past, it is questionable whether data older than that is relevant to making predictions going forward.

In [60]:
hockey_df = player_df[(player_df['GP'] >= 25) & (player_df['Year'] >= 2013)]

In [61]:
hockey_df.head()

,Player,EH_ID,API ID,Season,Team,Position,Shoots,Birthday,Age,Draft Yr,Draft Rd,Draft Ov,GP,TOI,GF%,SF%,FF%,CF%,xGF%,GF/60,GA/60,SF/60,SA/60,FF/60,FA/60,CF/60,CA/60,xGF/60,xGA/60,G±/60,S±/60,F±/60,C±/60,xG±/60,oiSh%,Sv%,xEVO_GAR/60,xEVD_GAR/60,xPPO_GAR/60,xSHD_GAR/60,xTake_GAR/60,xDraw_GAR/60,xOff_GAR/60,xDef_GAR/60,xPens_GAR/60,xGAR/60,xWAR/60,xSPAR/60,xEVO_GAR,xEVD_GAR,xPPO_GAR,xSHD_GAR,xTake_GAR,xDraw_GAR,xOff_GAR,xDef_GAR,xPens_GAR,xGAR,xWAR,xSPAR,G/60,A1/60,A2/60,Points/60,iSF/60,iFF/60,iCF/60,ixG/60,Sh%,FSh%,xFSh%,iBLK/60,GIVE/60,TAKE/60,iHF/60,iHA/60,iPENT2/60,iPEND2/60,iPENT5/60,iPEND5/60,iPEN±/60,FOW/60,FOL/60,FO±/60,G,A1,A2,Points,iSF,iFF,iCF,ixG,iBLK,GIVE,TAKE,iHF,iHA,iPENT2,iPEND2,iPENT5,iPEND5,iPEN±,FOW,FOL,FO±,GF,GA,SF,SA,FF,FA,CF,CA,xGF,xGA,G±,S±,F±,C±,xG±,TOI_All,TOI_EV,TOI_PP,TOI_SH,EVO_GAR,EVD_GAR,PPO_GAR,SHD_GAR,Take_GAR,Off_GAR,Def_GAR,GAR,WAR,SPAR,EVO_GAR/60,EVD_GAR/60,PPO_GAR/60,SHD_GAR/60,Take_GAR/60,Draw_GAR/60,Off_GAR/60,Def_GAR/60,Pens_GAR/60,GAR/60,WAR/60,SPAR/60,Year,First_Name,Last_Name,Player1,AGE,Cap_Hit,Salary
7,aaron ekblad,AARON.EKBLAD,8477932,14-15,FLA,D,R,1996-02-07,18,2014,1.0,1.0,81,1766.62,59.96,57.82,56.61,57.07,58.74,2.93,1.96,34.78,25.38,45.59,34.94,59.18,44.51,2.78,1.95,0.97,9.40,10.65,14.67,0.83,8.43,92.29,0.183,0.105,1.267,-0.170,0.033,0.023,0.324,0.100,0.056,0.454,0.086,0.165,4.5,2.6,4.7,-0.1,1.0,0.7,9.2,2.5,1.6,13.4,2.5,4.9,0.41,0.34,0.58,1.32,5.77,8.08,11.92,0.30,7.06,5.04,3.77,2.72,1.39,0.88,3.70,4.62,0.54,0.51,0.00,0.00,-0.03,0.0,0.0,0.0,12,10,17,39,170,238,351,8.97,80,41,26,109,136,16,15,0,0,-1,0,0,0,86.30,57.63,1024.10,747.21,1342.33,1028.68,1742.44,1310.56,81.86,57.49,28.67,276.89,313.65,431.88,24.37,1766.6,1485.7,221.4,31.5,5.5,5.7,1.2,-0.3,1.0,6.7,5.4,13.8,2.6,5.0,0.222,0.230,0.326,-0.547,0.033,0.023,0.236,0.214,0.056,0.467,0.089,0.170,2015,aar,ekblad,aaron ekblad,18.0,0.925,0.925
8,aaron ekblad,AARON.EKBLAD,8477932,15-16,FLA,D,R,1996-02-07,19,2014,1.0,1.0,78,1690.82,60.69,56.43,55.41,55.96,57.83,3.29,2.13,31.34,24.20,42.34,34.07,54.45,42.84,2.94,2.14,1.16,7.14,8.27,11.61,0.80,10.51,91.18,0.415,0.132,-0.004,0.359,0.034,-0.022,0.358,0.140,0.011,0.468,0.090,0.174,9.5,3.0,0.0,0.3,0.9,-0.6,9.5,3.4,0.3,13.2,2.5,4.9,0.53,0.28,0.46,1.28,6.46,8.52,11.50,0.36,8.24,6.25,4.25,2.09,1.45,0.85,3.09,4.47,0.67,0.28,0.04,0.04,-0.39,0.0,0.0,0.0,15,8,13,36,182,240,324,10.20,59,41,24,87,126,19,8,1,1,-11,0,0,0,92.85,60.15,883.09,681.85,1193.09,960.11,1534.30,1207.24,82.79,60.38,32.70,201.24,232.98,327.06,22.41,1690.8,1378.1,218.3,55.1,4.2,3.0,0.2,0.2,0.9,4.4,3.2,7.9,1.5,2.9,0.181,0.130,0.056,0.262,0.034,-0.022,0.164,0.135,0.011,0.280,0.054,0.104,2016,aar,ekblad,aaron ekblad,19.0,0.925,0.925
9,aaron ekblad,AARON.EKBLAD,8477932,16-17,FLA,D,R,1996-02-07,20,2014,1.0,1.0,68,1459.28,46.49,55.24,55.40,56.71,53.00,2.21,2.54,35.47,28.75,48.02,38.66,63.21,48.24,2.81,2.49,-0.33,6.72,9.36,14.96,0.32,6.22,91.17,-0.055,-0.007,0.449,0.754,-0.003,0.006,0.013,0.025,0.003,0.037,0.007,0.013,-1.1,-0.1,1.4,0.7,-0.1,0.1,0.3,0.5,0.1,0.9,0.2,0.3,0.41,0.16,0.29,0.86,9.25,12.17,16.78,0.49,4.44,3.38,4.06,2.55,2.38,1.03,3.62,3.45,1.19,0.70,0.00,0.00,-0.49,0.0,0.0,0.0,10,4,7,21,225,296,408,12.03,62,58,25,88,84,29,17,0,0,-12,0,0,0,53.64,61.75,862.68,699.13,1167.80,940.17,1537.28,1173.32,68.38,60.63,-8.11,163.55,227.63,363.96,7.75,1459.3,1194.7,187.3,52.9,-6.2,0.5,0.9,0.7,-0.1,-5.3,1.2,-4.0,-0.8,-1.5,-0.311,0.027,0.275,0.787,-0.003,0.006,-0.231,0.059,0.003,-0.166,-0.031,-0.060,2017,aar,ekblad,aaron ekblad,20.0,0.925,0.925
10,aaron ekblad,AARON.EKBLAD,8477932,17-18,FLA,D,R,1996-02-07,21,2014,1.0,1.0,82,1917.90,53.04,48.15,48.85,48.87,47.70,3.60,3.19,34.88,37.56,47.86,50.11,60.62,63.41,3.26,3.57,0.41,-2.68,-2.25,-2.79,-0.31,10.33,91.51,0.273,-0.197,1.128,-0.540,-0.036,-0.025,0.362,-0.234,-0.061,0.049,0.010,0.018,6.8,-4.9,3.3,-1.6,-1.2,-0.8,10.1,-6.6,-2.0,1.6,0.3,0.6,0.50,0.19,0.50,1.19,5.91,8.85,12.48,0.39,8.47,5.65,4.45,3.79,3.10,1.25,2.16,2.25,0.91,0.28,0.09,0.09,-0.63,0.0,0.0,0.0,16,6,16,38,189,283,399,12.60,121,99,40,69,7

In [62]:
hockey_df.reset_index(inplace = True)
hockey_df.drop(columns=['index'], inplace = True)

/Users/brianjohns/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [63]:
hockey_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6350 entries, 0 to 6349
Columns: 153 entries, Player to Salary
dtypes: datetime64[ns](1), float64(118), int64(25), object(9)
memory usage: 7.4+ MB


In [64]:
#A quick calculation to see the number of null values and the proportion of data it represents.
for col in hockey_df.columns:
    if hockey_df[col].isna().sum() > 0:
        nullvalues = hockey_df[col].isna().sum()
        shape = hockey_df.shape[0]
        print (f'{col}: {nullvalues} null values, {round(nullvalues/shape, 4)*100}% of the data')

Draft Rd: 731 null values, 11.51% of the data
Draft Ov: 731 null values, 11.51% of the data
Player1: 134 null values, 2.11% of the data
AGE: 134 null values, 2.11% of the data
Cap_Hit: 134 null values, 2.11% of the data
Salary: 134 null values, 2.11% of the data


The first columns I'll tackle will be about the draft information.

I am going to assume that the players who do not have draft information were players that were undrafted and therefore do not have a Draft Round or Draft Overall Pick associated with them.

However, the Evolving Hockey stats DOES have the draft YEAR that the player would have been drafted.  We'll look up the number of rounds and picks for each of those years.

In [65]:
hockey_df['Draft Rd'].value_counts()

1.0     2503
2.0     1041
3.0      557
4.0      470
5.0      366
6.0      344
7.0      241
9.0       56
8.0       38
10.0       2
11.0       1
Name: Draft Rd, dtype: int64

For draft rounds, the draft has had only 7 rounds since 2013, and looks as though that will be the case for the foreseeable future.  Therefore we are going bin all of the players drafted in the 8th round or later with the undrafted players to represent all players that would not have been draft in the first 7 rounds of any NHL draft.

In [66]:
hockey_df.loc[hockey_df['Draft Rd'].isna(), 'Draft Rd'] = 8
hockey_df.loc[hockey_df['Draft Rd'] == 9, 'Draft Rd'] = 8
hockey_df.loc[hockey_df['Draft Rd'] == 10, 'Draft Rd'] = 8
hockey_df.loc[hockey_df['Draft Rd'] == 11, 'Draft Rd'] = 8

/Users/brianjohns/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [67]:
hockey_df['Draft Rd'].value_counts()

1.0    2503
2.0    1041
8.0     828
3.0     557
4.0     470
5.0     366
6.0     344
7.0     241
Name: Draft Rd, dtype: int64

Similarly, we can put players who were undrafted after the last pick of the draft to fill in the null values and represent that these players would have been picked after the last pick of the draft.  We'll look at the draft
years for the players and can hard code the pick number for each year (as it changes from year to year).

In [68]:
hockey_df['Draft Yr'].loc[hockey_df['Draft Ov'].isna()].value_counts()

2008    90
2010    72
2009    69
2004    56
2002    53
2005    46
2012    46
2006    39
2011    36
2013    35
2003    34
2007    30
2001    29
1999    20
2014    18
1998    12
2000    11
1994     8
2015     8
2016     6
1997     5
1993     3
1996     3
1995     1
2017     1
Name: Draft Yr, dtype: int64

In [69]:
hockey_df.loc[(hockey_df['Draft Ov'].isna()) & (hockey_df['Draft Yr'] == 1993), 'Draft Ov'] = 287
hockey_df.loc[(hockey_df['Draft Ov'].isna()) & (hockey_df['Draft Yr'] == 1994), 'Draft Ov'] = 287
hockey_df.loc[(hockey_df['Draft Ov'].isna()) & (hockey_df['Draft Yr'] == 1995), 'Draft Ov'] = 235
hockey_df.loc[(hockey_df['Draft Ov'].isna()) & (hockey_df['Draft Yr'] == 1996), 'Draft Ov'] = 242
hockey_df.loc[(hockey_df['Draft Ov'].isna()) & (hockey_df['Draft Yr'] == 1997), 'Draft Ov'] = 247
hockey_df.loc[(hockey_df['Draft Ov'].isna()) & (hockey_df['Draft Yr'] == 1998), 'Draft Ov'] = 259
hockey_df.loc[(hockey_df['Draft Ov'].isna()) & (hockey_df['Draft Yr'] == 1999), 'Draft Ov'] = 273
hockey_df.loc[(hockey_df['Draft Ov'].isna()) & (hockey_df['Draft Yr'] == 2000), 'Draft Ov'] = 294
hockey_df.loc[(hockey_df['Draft Ov'].isna()) & (hockey_df['Draft Yr'] == 2001), 'Draft Ov'] = 290
hockey_df.loc[(hockey_df['Draft Ov'].isna()) & (hockey_df['Draft Yr'] == 2002), 'Draft Ov'] = 292
hockey_df.loc[(hockey_df['Draft Ov'].isna()) & (hockey_df['Draft Yr'] == 2003), 'Draft Ov'] = 293
hockey_df.loc[(hockey_df['Draft Ov'].isna()) & (hockey_df['Draft Yr'] == 2004), 'Draft Ov'] = 292
hockey_df.loc[(hockey_df['Draft Ov'].isna()) & (hockey_df['Draft Yr'] == 2005), 'Draft Ov'] = 231
hockey_df.loc[(hockey_df['Draft Ov'].isna()) & (hockey_df['Draft Yr'] == 2006), 'Draft Ov'] = 214
hockey_df.loc[(hockey_df['Draft Ov'].isna()) & (hockey_df['Draft Yr'] == 2007), 'Draft Ov'] = 212
hockey_df.loc[(hockey_df['Draft Ov'].isna()) & (hockey_df['Draft Yr'] == 2008), 'Draft Ov'] = 212
hockey_df.loc[(hockey_df['Draft Ov'].isna()) & (hockey_df['Draft Yr'] == 2009), 'Draft Ov'] = 211
hockey_df.loc[(hockey_df['Draft Ov'].isna()) & (hockey_df['Draft Yr'] == 2010), 'Draft Ov'] = 211
hockey_df.loc[(hockey_df['Draft Ov'].isna()) & (hockey_df['Draft Yr'] == 2011), 'Draft Ov'] = 212
hockey_df.loc[(hockey_df['Draft Ov'].isna()) & (hockey_df['Draft Yr'] == 2012), 'Draft Ov'] = 212
hockey_df.loc[(hockey_df['Draft Ov'].isna()) & (hockey_df['Draft Yr'] == 2013), 'Draft Ov'] = 212
hockey_df.loc[(hockey_df['Draft Ov'].isna()) & (hockey_df['Draft Yr'] == 2014), 'Draft Ov'] = 211
hockey_df.loc[(hockey_df['Draft Ov'].isna()) & (hockey_df['Draft Yr'] == 2015), 'Draft Ov'] = 212
hockey_df.loc[(hockey_df['Draft Ov'].isna()) & (hockey_df['Draft Yr'] == 2016), 'Draft Ov'] = 212
hockey_df.loc[(hockey_df['Draft Ov'].isna()) & (hockey_df['Draft Yr'] == 2017), 'Draft Ov'] = 218

In [70]:
#No more null values for Draft Pick Overall
hockey_df['Draft Yr'].loc[hockey_df['Draft Ov'].isna()].value_counts()

Series([], Name: Draft Yr, dtype: int64)

In [71]:
for col in hockey_df.columns:
    if hockey_df[col].isna().sum() > 0:
        nullvalues = hockey_df[col].isna().sum()
        shape = hockey_df.shape[0]
        print (f'{col}: {nullvalues} null values, {round(nullvalues/shape, 4)*100}% of the data')

Player1: 134 null values, 2.11% of the data
AGE: 134 null values, 2.11% of the data
Cap_Hit: 134 null values, 2.11% of the data
Salary: 134 null values, 2.11% of the data


Age and Player 1 is redundant with what is in the full dataset, so just Cap Hit and Salary have to be looked at.

The following list of players are likely players whose names did not perfectly match between Cap Friendly and Evolving Hockey Datasets

Referring to the CapFriendly website, we'll hard-code the salary information for the players who are missing values here.

In [72]:
hockey_df['Player'].loc[(hockey_df['Cap_Hit'].isna())].value_counts()

mike matheson       6
pa parenteau        5
jimmy hayes         5
martin st  louis    3
taylor pyatt        2
                   ..
deryk engelland     1
derek grant         1
derek forbort       1
dennis rasmussen    1
john moore          1
Name: Player, Length: 108, dtype: int64

In [73]:
hockey_df.loc[((hockey_df['Player']) == 'sam reinhart') & (hockey_df['Year'] == 2017), ['Cap_Hit', 'Salary']] = [.894, .832]
hockey_df.loc[((hockey_df['Player']) == 'troy terry') & (hockey_df['Year'] == 2019), ['Cap_Hit', 'Salary']] = [.925, .925]
hockey_df.loc[((hockey_df['Player']) == 'korbinian holzer') & (hockey_df['Year'] == 2020), ['Cap_Hit', 'Salary']] = [.85, .85]
hockey_df.loc[((hockey_df['Player']) == 'zach parise') & (hockey_df['Year'] == 2022), ['Cap_Hit', 'Salary']] = [7.538, 6.0]
hockey_df.loc[((hockey_df['Player']) == 'yannick weber') & (hockey_df['Year'] == 2017), ['Cap_Hit', 'Salary']] = [.575, .575]
hockey_df.loc[((hockey_df['Player']) == 'matt irwin') & (hockey_df['Year'] == 2017), ['Cap_Hit', 'Salary']] = [.575, .575]
hockey_df.loc[((hockey_df['Player']) == 'miikka salomaki') & (hockey_df['Year'] == 2020), ['Cap_Hit', 'Salary']] = [.75, .75]
hockey_df.loc[((hockey_df['Player']) == 'oscar fantenberg') & (hockey_df['Year'] == 2020), ['Cap_Hit', 'Salary']] = [.85, .85]
hockey_df.loc[((hockey_df['Player']) == 'klas dahlbeck') & (hockey_df['Year'] == 2018), ['Cap_Hit', 'Salary']] = [.850, .850]
hockey_df.loc[((hockey_df['Player']) == 'josh leivo') & (hockey_df['Year'] == 2019), ['Cap_Hit', 'Salary']] = [.925, .925]
hockey_df.loc[((hockey_df['Player']) == 'colby cave') & (hockey_df['Year'] == 2019), ['Cap_Hit', 'Salary']] = [.675, .675]
hockey_df.loc[((hockey_df['Player']) == 'andrei svechnikov') & (hockey_df['Year'] == 2019), ['Cap_Hit', 'Salary']] = [.925, .925]
hockey_df.loc[((hockey_df['Player']) == 'ben hutton') & (hockey_df['Year'] == 2022), ['Cap_Hit', 'Salary']] = [.925, .925]
hockey_df.loc[((hockey_df['Player']) == 'brendan lemieux') & (hockey_df['Year'] == 2020), ['Cap_Hit', 'Salary']] = [.925, .925]
hockey_df.loc[((hockey_df['Player']) == 'brian dumoulin') & (hockey_df['Year'] == 2019), ['Cap_Hit', 'Salary']] = [4.1, 4.1]
hockey_df.loc[((hockey_df['Player']) == 'chad ruhwedel') & (hockey_df['Year'] == 2017), ['Cap_Hit', 'Salary']] = [.575, .575]
hockey_df.loc[((hockey_df['Player']) == 'chris terry') & (hockey_df['Year'] == 2015), ['Cap_Hit', 'Salary']] = [.6, .6]
hockey_df.loc[((hockey_df['Player']) == 'conor sheary') & (hockey_df['Year'] == 2016), ['Cap_Hit', 'Salary']] = [.668, .668]
hockey_df.loc[((hockey_df['Player']) == 'john hayden') & (hockey_df['Year'] == 2018), ['Cap_Hit', 'Salary']] = [.925, .925]
hockey_df.loc[((hockey_df['Player']) == 'dennis rasmussen') & (hockey_df['Year'] == 2017), ['Cap_Hit', 'Salary']] = [.575, .575]
hockey_df.loc[((hockey_df['Player']) == 'gabriel dumont') & (hockey_df['Year'] == 2017), ['Cap_Hit', 'Salary']] = [.575, .575]
hockey_df.loc[((hockey_df['Player']) == 'gregory campbell') & (hockey_df['Year'] == 2015), ['Cap_Hit', 'Salary']] = [1.6, 1.7]
hockey_df.loc[((hockey_df['Player']) == 'james neal') & (hockey_df['Year'] == 2018), ['Cap_Hit', 'Salary']] = [5.0, 5.0]
hockey_df.loc[((hockey_df['Player']) == 'jason zucker') & (hockey_df['Year'] == 2022), ['Cap_Hit', 'Salary']] = [5.5, 6.25]
hockey_df.loc[((hockey_df['Player']) == 'jimmy vesey') & (hockey_df['Year'] == 2022), ['Cap_Hit', 'Salary']] = [.8, .8]
hockey_df.loc[((hockey_df['Player']) == 'joe pavelski') & (hockey_df['Year'] == 2015), ['Cap_Hit', 'Salary']] = [6.0, 6.0]

/Users/brianjohns/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1797: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, v, pi)


In [74]:
hockey_df.loc[((hockey_df['Player']) == 'pa parenteau') & (hockey_df['Year'] == 2013), ['Cap_Hit', 'Salary']] = [4, 4]
hockey_df.loc[((hockey_df['Player']) == 'pa parenteau') & (hockey_df['Year'] == 2014), ['Cap_Hit', 'Salary']] = [4, 4]
hockey_df.loc[((hockey_df['Player']) == 'pa parenteau') & (hockey_df['Year'] == 2015), ['Cap_Hit', 'Salary']] = [4, 4]
hockey_df.loc[((hockey_df['Player']) == 'pa parenteau') & (hockey_df['Year'] == 2016), ['Cap_Hit', 'Salary']] = [4, 4]
hockey_df.loc[((hockey_df['Player']) == 'pa parenteau') & (hockey_df['Year'] == 2017), ['Cap_Hit', 'Salary']] = [1.5, 1.5]

hockey_df.loc[((hockey_df['Player']) == 'erik karlsson') & (hockey_df['Year'] == 2015) & (hockey_df['Age'] == 19), 
              ['Player']] = ['erik karlsson2']
hockey_df.loc[((hockey_df['Player']) == 'erik karlsson') & (hockey_df['Year'] == 2016) & (hockey_df['Age'] == 20), 
              ['Player']] = ['erik karlsson2']
hockey_df.loc[((hockey_df['Player']) == 'erik karlsson') & (hockey_df['Year'] == 2017) & (hockey_df['Age'] == 21), 
              ['Player']] = ['erik karlsson2']

The remaining list of players actually did not have financial information available on the Cap Friendly website.  Without that information, unfortunately their datapoints are not helpful for the predictions of our models so we will drop the rest.

In [75]:
hockey_df['Player'].loc[(hockey_df['Cap_Hit'].isna())].value_counts()

mike matheson       6
jimmy hayes         5
martin st  louis    3
zenon konopka       2
henrik tallinder    2
                   ..
dylan demelo        1
dominik simon       1
deryk engelland     1
derek grant         1
keith aucoin        1
Name: Player, Length: 101, dtype: int64

In [76]:
hockey_df['Cap_Hit'].isna().sum()

123

In [77]:
hockey_df.dropna(axis = 0, subset = ['Cap_Hit'], inplace = True)

/Users/brianjohns/opt/anaconda3/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [78]:
hockey_df['Cap_Hit'].isna().sum()

0

In [79]:
#These are the remaining null values of the players that we hard-coded salaries for
#These columns will be dropped
for col in hockey_df.columns:
    if hockey_df[col].isna().sum() > 0:
        nullvalues = hockey_df[col].isna().sum()
        shape = hockey_df.shape[0]
        print (f'{col}: {nullvalues} null values, {round(nullvalues/shape, 4)*100}% of the data')

Player1: 11 null values, 0.18% of the data
AGE: 11 null values, 0.18% of the data


In [80]:
#All of these players now have valid information in all other categories so these columns
#Do not have any useful information.  These columns will be dropped eventually
hockey_df['Player'].loc[(hockey_df['AGE'].isna())].value_counts()

pa parenteau         5
andrei svechnikov    1
brian dumoulin       1
colby cave           1
dennis rasmussen     1
jason zucker         1
klas dahlbeck        1
Name: Player, dtype: int64

In [81]:
hockey_df.head()

,Player,EH_ID,API ID,Season,Team,Position,Shoots,Birthday,Age,Draft Yr,Draft Rd,Draft Ov,GP,TOI,GF%,SF%,FF%,CF%,xGF%,GF/60,GA/60,SF/60,SA/60,FF/60,FA/60,CF/60,CA/60,xGF/60,xGA/60,G±/60,S±/60,F±/60,C±/60,xG±/60,oiSh%,Sv%,xEVO_GAR/60,xEVD_GAR/60,xPPO_GAR/60,xSHD_GAR/60,xTake_GAR/60,xDraw_GAR/60,xOff_GAR/60,xDef_GAR/60,xPens_GAR/60,xGAR/60,xWAR/60,xSPAR/60,xEVO_GAR,xEVD_GAR,xPPO_GAR,xSHD_GAR,xTake_GAR,xDraw_GAR,xOff_GAR,xDef_GAR,xPens_GAR,xGAR,xWAR,xSPAR,G/60,A1/60,A2/60,Points/60,iSF/60,iFF/60,iCF/60,ixG/60,Sh%,FSh%,xFSh%,iBLK/60,GIVE/60,TAKE/60,iHF/60,iHA/60,iPENT2/60,iPEND2/60,iPENT5/60,iPEND5/60,iPEN±/60,FOW/60,FOL/60,FO±/60,G,A1,A2,Points,iSF,iFF,iCF,ixG,iBLK,GIVE,TAKE,iHF,iHA,iPENT2,iPEND2,iPENT5,iPEND5,iPEN±,FOW,FOL,FO±,GF,GA,SF,SA,FF,FA,CF,CA,xGF,xGA,G±,S±,F±,C±,xG±,TOI_All,TOI_EV,TOI_PP,TOI_SH,EVO_GAR,EVD_GAR,PPO_GAR,SHD_GAR,Take_GAR,Off_GAR,Def_GAR,GAR,WAR,SPAR,EVO_GAR/60,EVD_GAR/60,PPO_GAR/60,SHD_GAR/60,Take_GAR/60,Draw_GAR/60,Off_GAR/60,Def_GAR/60,Pens_GAR/60,GAR/60,WAR/60,SPAR/60,Year,First_Name,Last_Name,Player1,AGE,Cap_Hit,Salary
0,aaron ekblad,AARON.EKBLAD,8477932,14-15,FLA,D,R,1996-02-07,18,2014,1.0,1.0,81,1766.62,59.96,57.82,56.61,57.07,58.74,2.93,1.96,34.78,25.38,45.59,34.94,59.18,44.51,2.78,1.95,0.97,9.40,10.65,14.67,0.83,8.43,92.29,0.183,0.105,1.267,-0.170,0.033,0.023,0.324,0.100,0.056,0.454,0.086,0.165,4.5,2.6,4.7,-0.1,1.0,0.7,9.2,2.5,1.6,13.4,2.5,4.9,0.41,0.34,0.58,1.32,5.77,8.08,11.92,0.30,7.06,5.04,3.77,2.72,1.39,0.88,3.70,4.62,0.54,0.51,0.00,0.00,-0.03,0.0,0.0,0.0,12,10,17,39,170,238,351,8.97,80,41,26,109,136,16,15,0,0,-1,0,0,0,86.30,57.63,1024.10,747.21,1342.33,1028.68,1742.44,1310.56,81.86,57.49,28.67,276.89,313.65,431.88,24.37,1766.6,1485.7,221.4,31.5,5.5,5.7,1.2,-0.3,1.0,6.7,5.4,13.8,2.6,5.0,0.222,0.230,0.326,-0.547,0.033,0.023,0.236,0.214,0.056,0.467,0.089,0.170,2015,aar,ekblad,aaron ekblad,18.0,0.925,0.925
1,aaron ekblad,AARON.EKBLAD,8477932,15-16,FLA,D,R,1996-02-07,19,2014,1.0,1.0,78,1690.82,60.69,56.43,55.41,55.96,57.83,3.29,2.13,31.34,24.20,42.34,34.07,54.45,42.84,2.94,2.14,1.16,7.14,8.27,11.61,0.80,10.51,91.18,0.415,0.132,-0.004,0.359,0.034,-0.022,0.358,0.140,0.011,0.468,0.090,0.174,9.5,3.0,0.0,0.3,0.9,-0.6,9.5,3.4,0.3,13.2,2.5,4.9,0.53,0.28,0.46,1.28,6.46,8.52,11.50,0.36,8.24,6.25,4.25,2.09,1.45,0.85,3.09,4.47,0.67,0.28,0.04,0.04,-0.39,0.0,0.0,0.0,15,8,13,36,182,240,324,10.20,59,41,24,87,126,19,8,1,1,-11,0,0,0,92.85,60.15,883.09,681.85,1193.09,960.11,1534.30,1207.24,82.79,60.38,32.70,201.24,232.98,327.06,22.41,1690.8,1378.1,218.3,55.1,4.2,3.0,0.2,0.2,0.9,4.4,3.2,7.9,1.5,2.9,0.181,0.130,0.056,0.262,0.034,-0.022,0.164,0.135,0.011,0.280,0.054,0.104,2016,aar,ekblad,aaron ekblad,19.0,0.925,0.925
2,aaron ekblad,AARON.EKBLAD,8477932,16-17,FLA,D,R,1996-02-07,20,2014,1.0,1.0,68,1459.28,46.49,55.24,55.40,56.71,53.00,2.21,2.54,35.47,28.75,48.02,38.66,63.21,48.24,2.81,2.49,-0.33,6.72,9.36,14.96,0.32,6.22,91.17,-0.055,-0.007,0.449,0.754,-0.003,0.006,0.013,0.025,0.003,0.037,0.007,0.013,-1.1,-0.1,1.4,0.7,-0.1,0.1,0.3,0.5,0.1,0.9,0.2,0.3,0.41,0.16,0.29,0.86,9.25,12.17,16.78,0.49,4.44,3.38,4.06,2.55,2.38,1.03,3.62,3.45,1.19,0.70,0.00,0.00,-0.49,0.0,0.0,0.0,10,4,7,21,225,296,408,12.03,62,58,25,88,84,29,17,0,0,-12,0,0,0,53.64,61.75,862.68,699.13,1167.80,940.17,1537.28,1173.32,68.38,60.63,-8.11,163.55,227.63,363.96,7.75,1459.3,1194.7,187.3,52.9,-6.2,0.5,0.9,0.7,-0.1,-5.3,1.2,-4.0,-0.8,-1.5,-0.311,0.027,0.275,0.787,-0.003,0.006,-0.231,0.059,0.003,-0.166,-0.031,-0.060,2017,aar,ekblad,aaron ekblad,20.0,0.925,0.925
3,aaron ekblad,AARON.EKBLAD,8477932,17-18,FLA,D,R,1996-02-07,21,2014,1.0,1.0,82,1917.90,53.04,48.15,48.85,48.87,47.70,3.60,3.19,34.88,37.56,47.86,50.11,60.62,63.41,3.26,3.57,0.41,-2.68,-2.25,-2.79,-0.31,10.33,91.51,0.273,-0.197,1.128,-0.540,-0.036,-0.025,0.362,-0.234,-0.061,0.049,0.010,0.018,6.8,-4.9,3.3,-1.6,-1.2,-0.8,10.1,-6.6,-2.0,1.6,0.3,0.6,0.50,0.19,0.50,1.19,5.91,8.85,12.48,0.39,8.47,5.65,4.45,3.79,3.10,1.25,2.16,2.25,0.91,0.28,0.09,0.09,-0.63,0.0,0.0,0.0,16,6,16,38,189,283,399,12.60,121,99,40,69,72

To get more use of some of our biogrpahical information that could be useful, we will transform them into numeric values.  First, we will keep the handedness of the player by binarizing the data and making Left-Handed = 1

In [82]:
#We are going to keep the handedness of the player and binarize the data
#Most players are Lefthanded, so we'll make that the column name
hockey_df['Left_Handed'] = np.where(hockey_df['Shoots'] == 'L', 1, 0)

/var/folders/dm/5cg2631s7057bxm5mx52z5fr0000gn/T/ipykernel_71194/3019850826.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hockey_df['Left_Handed'] = np.where(hockey_df['Shoots'] == 'L', 1, 0)


In [83]:
hockey_df.head()

,Player,EH_ID,API ID,Season,Team,Position,Shoots,Birthday,Age,Draft Yr,Draft Rd,Draft Ov,GP,TOI,GF%,SF%,FF%,CF%,xGF%,GF/60,GA/60,SF/60,SA/60,FF/60,FA/60,CF/60,CA/60,xGF/60,xGA/60,G±/60,S±/60,F±/60,C±/60,xG±/60,oiSh%,Sv%,xEVO_GAR/60,xEVD_GAR/60,xPPO_GAR/60,xSHD_GAR/60,xTake_GAR/60,xDraw_GAR/60,xOff_GAR/60,xDef_GAR/60,xPens_GAR/60,xGAR/60,xWAR/60,xSPAR/60,xEVO_GAR,xEVD_GAR,xPPO_GAR,xSHD_GAR,xTake_GAR,xDraw_GAR,xOff_GAR,xDef_GAR,xPens_GAR,xGAR,xWAR,xSPAR,G/60,A1/60,A2/60,Points/60,iSF/60,iFF/60,iCF/60,ixG/60,Sh%,FSh%,xFSh%,iBLK/60,GIVE/60,TAKE/60,iHF/60,iHA/60,iPENT2/60,iPEND2/60,iPENT5/60,iPEND5/60,iPEN±/60,FOW/60,FOL/60,FO±/60,G,A1,A2,Points,iSF,iFF,iCF,ixG,iBLK,GIVE,TAKE,iHF,iHA,iPENT2,iPEND2,iPENT5,iPEND5,iPEN±,FOW,FOL,FO±,GF,GA,SF,SA,FF,FA,CF,CA,xGF,xGA,G±,S±,F±,C±,xG±,TOI_All,TOI_EV,TOI_PP,TOI_SH,EVO_GAR,EVD_GAR,PPO_GAR,SHD_GAR,Take_GAR,Off_GAR,Def_GAR,GAR,WAR,SPAR,EVO_GAR/60,EVD_GAR/60,PPO_GAR/60,SHD_GAR/60,Take_GAR/60,Draw_GAR/60,Off_GAR/60,Def_GAR/60,Pens_GAR/60,GAR/60,WAR/60,SPAR/60,Year,First_Name,Last_Name,Player1,AGE,Cap_Hit,Salary,Left_Handed
0,aaron ekblad,AARON.EKBLAD,8477932,14-15,FLA,D,R,1996-02-07,18,2014,1.0,1.0,81,1766.62,59.96,57.82,56.61,57.07,58.74,2.93,1.96,34.78,25.38,45.59,34.94,59.18,44.51,2.78,1.95,0.97,9.40,10.65,14.67,0.83,8.43,92.29,0.183,0.105,1.267,-0.170,0.033,0.023,0.324,0.100,0.056,0.454,0.086,0.165,4.5,2.6,4.7,-0.1,1.0,0.7,9.2,2.5,1.6,13.4,2.5,4.9,0.41,0.34,0.58,1.32,5.77,8.08,11.92,0.30,7.06,5.04,3.77,2.72,1.39,0.88,3.70,4.62,0.54,0.51,0.00,0.00,-0.03,0.0,0.0,0.0,12,10,17,39,170,238,351,8.97,80,41,26,109,136,16,15,0,0,-1,0,0,0,86.30,57.63,1024.10,747.21,1342.33,1028.68,1742.44,1310.56,81.86,57.49,28.67,276.89,313.65,431.88,24.37,1766.6,1485.7,221.4,31.5,5.5,5.7,1.2,-0.3,1.0,6.7,5.4,13.8,2.6,5.0,0.222,0.230,0.326,-0.547,0.033,0.023,0.236,0.214,0.056,0.467,0.089,0.170,2015,aar,ekblad,aaron ekblad,18.0,0.925,0.925,0
1,aaron ekblad,AARON.EKBLAD,8477932,15-16,FLA,D,R,1996-02-07,19,2014,1.0,1.0,78,1690.82,60.69,56.43,55.41,55.96,57.83,3.29,2.13,31.34,24.20,42.34,34.07,54.45,42.84,2.94,2.14,1.16,7.14,8.27,11.61,0.80,10.51,91.18,0.415,0.132,-0.004,0.359,0.034,-0.022,0.358,0.140,0.011,0.468,0.090,0.174,9.5,3.0,0.0,0.3,0.9,-0.6,9.5,3.4,0.3,13.2,2.5,4.9,0.53,0.28,0.46,1.28,6.46,8.52,11.50,0.36,8.24,6.25,4.25,2.09,1.45,0.85,3.09,4.47,0.67,0.28,0.04,0.04,-0.39,0.0,0.0,0.0,15,8,13,36,182,240,324,10.20,59,41,24,87,126,19,8,1,1,-11,0,0,0,92.85,60.15,883.09,681.85,1193.09,960.11,1534.30,1207.24,82.79,60.38,32.70,201.24,232.98,327.06,22.41,1690.8,1378.1,218.3,55.1,4.2,3.0,0.2,0.2,0.9,4.4,3.2,7.9,1.5,2.9,0.181,0.130,0.056,0.262,0.034,-0.022,0.164,0.135,0.011,0.280,0.054,0.104,2016,aar,ekblad,aaron ekblad,19.0,0.925,0.925,0
2,aaron ekblad,AARON.EKBLAD,8477932,16-17,FLA,D,R,1996-02-07,20,2014,1.0,1.0,68,1459.28,46.49,55.24,55.40,56.71,53.00,2.21,2.54,35.47,28.75,48.02,38.66,63.21,48.24,2.81,2.49,-0.33,6.72,9.36,14.96,0.32,6.22,91.17,-0.055,-0.007,0.449,0.754,-0.003,0.006,0.013,0.025,0.003,0.037,0.007,0.013,-1.1,-0.1,1.4,0.7,-0.1,0.1,0.3,0.5,0.1,0.9,0.2,0.3,0.41,0.16,0.29,0.86,9.25,12.17,16.78,0.49,4.44,3.38,4.06,2.55,2.38,1.03,3.62,3.45,1.19,0.70,0.00,0.00,-0.49,0.0,0.0,0.0,10,4,7,21,225,296,408,12.03,62,58,25,88,84,29,17,0,0,-12,0,0,0,53.64,61.75,862.68,699.13,1167.80,940.17,1537.28,1173.32,68.38,60.63,-8.11,163.55,227.63,363.96,7.75,1459.3,1194.7,187.3,52.9,-6.2,0.5,0.9,0.7,-0.1,-5.3,1.2,-4.0,-0.8,-1.5,-0.311,0.027,0.275,0.787,-0.003,0.006,-0.231,0.059,0.003,-0.166,-0.031,-0.060,2017,aar,ekblad,aaron ekblad,20.0,0.925,0.925,0
3,aaron ekblad,AARON.EKBLAD,8477932,17-18,FLA,D,R,1996-02-07,21,2014,1.0,1.0,82,1917.90,53.04,48.15,48.85,48.87,47.70,3.60,3.19,34.88,37.56,47.86,50.11,60.62,63.41,3.26,3.57,0.41,-2.68,-2.25,-2.79,-0.31,10.33,91.51,0.273,-0.197,1.128,-0.540,-0.036,-0.025,0.362,-0.234,-0.061,0.049,0.010,0.018,6.8,-4.9,3.3,-1.6,-1.2,-0.8,10.1,-6.6,-2.0,1.6,0.3,0.6,0.50,0.19,0.50,1.19,5.91,8.85,12.48,0.39,8.47,5.65,4.45,3.79,3.10,1.25,2.16,2.25,0.91,0.28,0.09,0.09,-0.63,0.0,0.0,0.0,16,6,16,38,189,283,399,12.

We have a handful of players that are listed with multiple positions.  These are all players that play multiple forward positions (Center, Left Wing and Right Wing).

There are no defensemen that have played forward positions in this list, or vice versa.

Therefore, we will relabel these players F for Forwards, and then binarize the positions into Defencemen (1) and Forwards (0).

In [84]:
hockey_df[hockey_df['Position'].str.contains('/')]

,Player,EH_ID,API ID,Season,Team,Position,Shoots,Birthday,Age,Draft Yr,Draft Rd,Draft Ov,GP,TOI,GF%,SF%,FF%,CF%,xGF%,GF/60,GA/60,SF/60,SA/60,FF/60,FA/60,CF/60,CA/60,xGF/60,xGA/60,G±/60,S±/60,F±/60,C±/60,xG±/60,oiSh%,Sv%,xEVO_GAR/60,xEVD_GAR/60,xPPO_GAR/60,xSHD_GAR/60,xTake_GAR/60,xDraw_GAR/60,xOff_GAR/60,xDef_GAR/60,xPens_GAR/60,xGAR/60,xWAR/60,xSPAR/60,xEVO_GAR,xEVD_GAR,xPPO_GAR,xSHD_GAR,xTake_GAR,xDraw_GAR,xOff_GAR,xDef_GAR,xPens_GAR,xGAR,xWAR,xSPAR,G/60,A1/60,A2/60,Points/60,iSF/60,iFF/60,iCF/60,ixG/60,Sh%,FSh%,xFSh%,iBLK/60,GIVE/60,TAKE/60,iHF/60,iHA/60,iPENT2/60,iPEND2/60,iPENT5/60,iPEND5/60,iPEN±/60,FOW/60,FOL/60,FO±/60,G,A1,A2,Points,iSF,iFF,iCF,ixG,iBLK,GIVE,TAKE,iHF,iHA,iPENT2,iPEND2,iPENT5,iPEND5,iPEN±,FOW,FOL,FO±,GF,GA,SF,SA,FF,FA,CF,CA,xGF,xGA,G±,S±,F±,C±,xG±,TOI_All,TOI_EV,TOI_PP,TOI_SH,EVO_GAR,EVD_GAR,PPO_GAR,SHD_GAR,Take_GAR,Off_GAR,Def_GAR,GAR,WAR,SPAR,EVO_GAR/60,EVD_GAR/60,PPO_GAR/60,SHD_GAR/60,Take_GAR/60,Draw_GAR/60,Off_GAR/60,Def_GAR/60,Pens_GAR/60,GAR/60,WAR/60,SPAR/60,Year,First_Name,Last_Name,Player1,AGE,Cap_Hit,Salary,Left_Handed
62,adam lowry,ADAM.LOWRY,8476392,21-22,WPG,C/L,L,1993-03-29,28,2011,3.0,67.0,53,811.95,39.45,45.39,45.49,46.04,47.26,1.91,2.92,27.05,32.54,37.12,44.48,49.45,57.96,2.64,2.94,-1.02,-5.49,-7.35,-8.51,-0.31,7.05,91.01,-0.098,0.145,0.625,0.140,0.054,0.059,-0.059,0.145,0.114,0.200,0.036,0.068,-1.1,1.6,0.4,0.3,0.7,0.8,-0.7,1.8,1.5,2.7,0.5,0.9,0.44,0.30,0.15,0.89,4.58,6.58,8.28,0.71,9.68,6.74,10.73,1.85,1.03,2.14,11.45,5.54,0.44,0.96,0.44,0.44,0.52,26.38,22.98,3.40,6,4,2,12,62,89,112,9.55,25,14,29,155,75,6,13,6,6,7,357,311,46,25.79,39.58,366.03,440.34,502.35,601.88,669.16,784.30,35.66,39.80,-13.79,-74.31,-99.53,-115.14,-4.14,812.0,642.1,36.5,119.0,-5.1,2.6,-0.5,0.6,0.7,-5.7,3.3,-0.9,-0.2,-0.3,-0.479,0.244,-0.863,0.326,0.054,0.059,-0.500,0.257,0.114,-0.063,-0.011,-0.021,2022,ada,lowry,adam lowry,28.0,3.250,2.500,1
84,adrian kempe,ADRIAN.KEMPE,8477960,21-22,L.A,C/L,L,1996-09-13,25,2014,1.0,29.0,51,933.42,53.61,56.79,57.11,57.83,52.94,3.61,3.13,38.02,28.93,51.47,38.65,67.84,49.47,3.76,3.34,0.49,9.09,12.82,18.38,0.42,9.51,89.19,0.639,-0.277,0.146,-1.693,-0.011,0.041,0.548,-0.420,0.030,0.179,0.032,0.061,7.2,-3.1,0.4,-2.1,-0.2,0.6,7.6,-5.3,0.5,2.8,0.5,0.9,1.61,0.19,0.51,2.31,9.77,13.82,18.38,1.29,16.45,11.63,9.34,0.96,1.67,1.61,3.99,2.96,0.96,1.03,0.00,0.00,0.06,1.22,1.16,0.06,25,3,8,36,152,215,286,20.09,15,26,25,62,46,15,16,0,0,1,19,18,1,56.23,48.65,591.55,450.13,800.72,601.25,1055.44,769.56,58.42,51.93,7.58,141.42,199.47,285.88,6.49,933.4,678.8,153.2,76.1,5.8,-2.6,0.6,-2.1,-0.2,6.4,-4.7,2.1,0.4,0.7,0.510,-0.232,0.234,-1.630,-0.011,0.041,0.459,-0.373,0.030,0.137,0.025,0.047,2022,adr,kempe,adrian kempe,24.0,2.000,2.500,1
137,alex debrincat,ALEX.DEBRINCAT,8479337,21-22,CHI,L/R,R,1997-12-18,23,2016,2.0,39.0,54,1100.23,51.77,55.61,53.79,53.62,56.61,3.83,3.57,35.14,28.06,44.93,38.60,60.23,52.10,3.56,2.73,0.26,7.09,6.34,8.14,0.83,10.91,87.27,0.638,-0.104,1.790,1.537,0.094,0.039,0.860,-0.023,0.132,0.911,0.163,0.309,8.7,-1.4,5.9,1.1,1.7,0.7,14.6,-0.3,2.4,16.7,3.0,5.7,1.58,0.60,0.27,2.45,9.49,12.16,16.63,1.15,16.67,13.00,9.46,1.25,1.69,2.13,3.65,3.93,0.22,0.93,0.05,0.05,0.71,0.71,1.42,-0.71,29,11,5,45,174,223,305,21.09,23,31,39,67,72,4,17,1,1,13,13,26,-13,70.28,65.47,644.44,514.45,823.98,707.73,1104.51,955.30,65.27,50.02,4.81,129.99,116.25,149.21,15.25,1100.2,822.7,196.1,43.1,4.0,-0.9,3.5,0.9,1.7,7.5,0.0,10.0,1.8,3.4,0.293,-0.064,1.077,1.278,0.094,0.039,0.444,0.003,0.132,0.545,0.098,0.185,2022,ale,debrincat,alex debrincat,23.0,6.400,5.100,0
188,alex killorn,ALEX.KILLORN,8473986,21-22,T.B,C/L,L,1989-09-14,32,2007,3.0,77.0,51,976.75,59.12,53.19,54.20,54.03,53.60,4.49,3.11,33.07,29.11,46.86,39.60,60.07,51.11,3.66,3.17,1.39,3.96,7.26,8.96,0.49,13.59,89.33,0.536,-0.206,0.417,0.376,-0.070,0.030,0.512,-0.143,-0.041,0.296,0.053,0.100,6.2,-2.4,1.2,0.5,-1.1,0.5,7.3,-1.8,-0.7,4.8,0.9,1.6,0.98,0.98,0.80,2.76,6.14,8.17,9.95,0.85,16.00,12.03,10.46,0.98,0.98,1.72,3.44,4.48,1.29,1.04,0.00,0.00,-0.25,0.00,0.43,-0.43

In [85]:
hockey_df.loc[hockey_df['Position'].str.contains('/'), 'Position'] = 'F'

/Users/brianjohns/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [86]:
hockey_df['Defenceman'] = np.where(hockey_df['Position'] == 'D', 1, 0)

/var/folders/dm/5cg2631s7057bxm5mx52z5fr0000gn/T/ipykernel_71194/942888318.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hockey_df['Defenceman'] = np.where(hockey_df['Position'] == 'D', 1, 0)


In [87]:
hockey_df.head()

,Player,EH_ID,API ID,Season,Team,Position,Shoots,Birthday,Age,Draft Yr,Draft Rd,Draft Ov,GP,TOI,GF%,SF%,FF%,CF%,xGF%,GF/60,GA/60,SF/60,SA/60,FF/60,FA/60,CF/60,CA/60,xGF/60,xGA/60,G±/60,S±/60,F±/60,C±/60,xG±/60,oiSh%,Sv%,xEVO_GAR/60,xEVD_GAR/60,xPPO_GAR/60,xSHD_GAR/60,xTake_GAR/60,xDraw_GAR/60,xOff_GAR/60,xDef_GAR/60,xPens_GAR/60,xGAR/60,xWAR/60,xSPAR/60,xEVO_GAR,xEVD_GAR,xPPO_GAR,xSHD_GAR,xTake_GAR,xDraw_GAR,xOff_GAR,xDef_GAR,xPens_GAR,xGAR,xWAR,xSPAR,G/60,A1/60,A2/60,Points/60,iSF/60,iFF/60,iCF/60,ixG/60,Sh%,FSh%,xFSh%,iBLK/60,GIVE/60,TAKE/60,iHF/60,iHA/60,iPENT2/60,iPEND2/60,iPENT5/60,iPEND5/60,iPEN±/60,FOW/60,FOL/60,FO±/60,G,A1,A2,Points,iSF,iFF,iCF,ixG,iBLK,GIVE,TAKE,iHF,iHA,iPENT2,iPEND2,iPENT5,iPEND5,iPEN±,FOW,FOL,FO±,GF,GA,SF,SA,FF,FA,CF,CA,xGF,xGA,G±,S±,F±,C±,xG±,TOI_All,TOI_EV,TOI_PP,TOI_SH,EVO_GAR,EVD_GAR,PPO_GAR,SHD_GAR,Take_GAR,Off_GAR,Def_GAR,GAR,WAR,SPAR,EVO_GAR/60,EVD_GAR/60,PPO_GAR/60,SHD_GAR/60,Take_GAR/60,Draw_GAR/60,Off_GAR/60,Def_GAR/60,Pens_GAR/60,GAR/60,WAR/60,SPAR/60,Year,First_Name,Last_Name,Player1,AGE,Cap_Hit,Salary,Left_Handed,Defenceman
0,aaron ekblad,AARON.EKBLAD,8477932,14-15,FLA,D,R,1996-02-07,18,2014,1.0,1.0,81,1766.62,59.96,57.82,56.61,57.07,58.74,2.93,1.96,34.78,25.38,45.59,34.94,59.18,44.51,2.78,1.95,0.97,9.40,10.65,14.67,0.83,8.43,92.29,0.183,0.105,1.267,-0.170,0.033,0.023,0.324,0.100,0.056,0.454,0.086,0.165,4.5,2.6,4.7,-0.1,1.0,0.7,9.2,2.5,1.6,13.4,2.5,4.9,0.41,0.34,0.58,1.32,5.77,8.08,11.92,0.30,7.06,5.04,3.77,2.72,1.39,0.88,3.70,4.62,0.54,0.51,0.00,0.00,-0.03,0.0,0.0,0.0,12,10,17,39,170,238,351,8.97,80,41,26,109,136,16,15,0,0,-1,0,0,0,86.30,57.63,1024.10,747.21,1342.33,1028.68,1742.44,1310.56,81.86,57.49,28.67,276.89,313.65,431.88,24.37,1766.6,1485.7,221.4,31.5,5.5,5.7,1.2,-0.3,1.0,6.7,5.4,13.8,2.6,5.0,0.222,0.230,0.326,-0.547,0.033,0.023,0.236,0.214,0.056,0.467,0.089,0.170,2015,aar,ekblad,aaron ekblad,18.0,0.925,0.925,0,1
1,aaron ekblad,AARON.EKBLAD,8477932,15-16,FLA,D,R,1996-02-07,19,2014,1.0,1.0,78,1690.82,60.69,56.43,55.41,55.96,57.83,3.29,2.13,31.34,24.20,42.34,34.07,54.45,42.84,2.94,2.14,1.16,7.14,8.27,11.61,0.80,10.51,91.18,0.415,0.132,-0.004,0.359,0.034,-0.022,0.358,0.140,0.011,0.468,0.090,0.174,9.5,3.0,0.0,0.3,0.9,-0.6,9.5,3.4,0.3,13.2,2.5,4.9,0.53,0.28,0.46,1.28,6.46,8.52,11.50,0.36,8.24,6.25,4.25,2.09,1.45,0.85,3.09,4.47,0.67,0.28,0.04,0.04,-0.39,0.0,0.0,0.0,15,8,13,36,182,240,324,10.20,59,41,24,87,126,19,8,1,1,-11,0,0,0,92.85,60.15,883.09,681.85,1193.09,960.11,1534.30,1207.24,82.79,60.38,32.70,201.24,232.98,327.06,22.41,1690.8,1378.1,218.3,55.1,4.2,3.0,0.2,0.2,0.9,4.4,3.2,7.9,1.5,2.9,0.181,0.130,0.056,0.262,0.034,-0.022,0.164,0.135,0.011,0.280,0.054,0.104,2016,aar,ekblad,aaron ekblad,19.0,0.925,0.925,0,1
2,aaron ekblad,AARON.EKBLAD,8477932,16-17,FLA,D,R,1996-02-07,20,2014,1.0,1.0,68,1459.28,46.49,55.24,55.40,56.71,53.00,2.21,2.54,35.47,28.75,48.02,38.66,63.21,48.24,2.81,2.49,-0.33,6.72,9.36,14.96,0.32,6.22,91.17,-0.055,-0.007,0.449,0.754,-0.003,0.006,0.013,0.025,0.003,0.037,0.007,0.013,-1.1,-0.1,1.4,0.7,-0.1,0.1,0.3,0.5,0.1,0.9,0.2,0.3,0.41,0.16,0.29,0.86,9.25,12.17,16.78,0.49,4.44,3.38,4.06,2.55,2.38,1.03,3.62,3.45,1.19,0.70,0.00,0.00,-0.49,0.0,0.0,0.0,10,4,7,21,225,296,408,12.03,62,58,25,88,84,29,17,0,0,-12,0,0,0,53.64,61.75,862.68,699.13,1167.80,940.17,1537.28,1173.32,68.38,60.63,-8.11,163.55,227.63,363.96,7.75,1459.3,1194.7,187.3,52.9,-6.2,0.5,0.9,0.7,-0.1,-5.3,1.2,-4.0,-0.8,-1.5,-0.311,0.027,0.275,0.787,-0.003,0.006,-0.231,0.059,0.003,-0.166,-0.031,-0.060,2017,aar,ekblad,aaron ekblad,20.0,0.925,0.925,0,1
3,aaron ekblad,AARON.EKBLAD,8477932,17-18,FLA,D,R,1996-02-07,21,2014,1.0,1.0,82,1917.90,53.04,48.15,48.85,48.87,47.70,3.60,3.19,34.88,37.56,47.86,50.11,60.62,63.41,3.26,3.57,0.41,-2.68,-2.25,-2.79,-0.31,10.33,91.51,0.273,-0.197,1.128,-0.540,-0.036,-0.025,0.362,-0.234,-0.061,0.049,0.010,0.018,6.8,-4.9,3.3,-1.6,-1.2,-0.8,10.1,-6.6,-2.0,1.6,0.3,0.6,0.50,0.19,0.50,1.19,5.91,8.85,12.48,0.39,8.47,5.65,4.45,3.79,3.10,1.25,2.16,2.25,0.91,0.28,0.09,0.09,-0.63,0.0,0.0,0.0,16,6,16,3

Now we will drop unnecessary columns that we will not be using.  This includes columns that are duplicative but under different names, remnants of our now binarized columns and portions of ID that were used solely to merge data.

In [88]:
hockey_df.drop(columns=['EH_ID', 'API ID', 'Season', 'Shoots', 'AGE', 'Player1', 'First_Name', 'Last_Name', 'Position'],
               axis = 1, inplace = True)

/Users/brianjohns/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [89]:
hockey_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6227 entries, 0 to 6347
Columns: 146 entries, Player to Defenceman
dtypes: datetime64[ns](1), float64(117), int64(26), object(2)
memory usage: 7.0+ MB


In [90]:
hockey_df.head()

,Player,Team,Birthday,Age,Draft Yr,Draft Rd,Draft Ov,GP,TOI,GF%,SF%,FF%,CF%,xGF%,GF/60,GA/60,SF/60,SA/60,FF/60,FA/60,CF/60,CA/60,xGF/60,xGA/60,G±/60,S±/60,F±/60,C±/60,xG±/60,oiSh%,Sv%,xEVO_GAR/60,xEVD_GAR/60,xPPO_GAR/60,xSHD_GAR/60,xTake_GAR/60,xDraw_GAR/60,xOff_GAR/60,xDef_GAR/60,xPens_GAR/60,xGAR/60,xWAR/60,xSPAR/60,xEVO_GAR,xEVD_GAR,xPPO_GAR,xSHD_GAR,xTake_GAR,xDraw_GAR,xOff_GAR,xDef_GAR,xPens_GAR,xGAR,xWAR,xSPAR,G/60,A1/60,A2/60,Points/60,iSF/60,iFF/60,iCF/60,ixG/60,Sh%,FSh%,xFSh%,iBLK/60,GIVE/60,TAKE/60,iHF/60,iHA/60,iPENT2/60,iPEND2/60,iPENT5/60,iPEND5/60,iPEN±/60,FOW/60,FOL/60,FO±/60,G,A1,A2,Points,iSF,iFF,iCF,ixG,iBLK,GIVE,TAKE,iHF,iHA,iPENT2,iPEND2,iPENT5,iPEND5,iPEN±,FOW,FOL,FO±,GF,GA,SF,SA,FF,FA,CF,CA,xGF,xGA,G±,S±,F±,C±,xG±,TOI_All,TOI_EV,TOI_PP,TOI_SH,EVO_GAR,EVD_GAR,PPO_GAR,SHD_GAR,Take_GAR,Off_GAR,Def_GAR,GAR,WAR,SPAR,EVO_GAR/60,EVD_GAR/60,PPO_GAR/60,SHD_GAR/60,Take_GAR/60,Draw_GAR/60,Off_GAR/60,Def_GAR/60,Pens_GAR/60,GAR/60,WAR/60,SPAR/60,Year,Cap_Hit,Salary,Left_Handed,Defenceman
0,aaron ekblad,FLA,1996-02-07,18,2014,1.0,1.0,81,1766.62,59.96,57.82,56.61,57.07,58.74,2.93,1.96,34.78,25.38,45.59,34.94,59.18,44.51,2.78,1.95,0.97,9.40,10.65,14.67,0.83,8.43,92.29,0.183,0.105,1.267,-0.170,0.033,0.023,0.324,0.100,0.056,0.454,0.086,0.165,4.5,2.6,4.7,-0.1,1.0,0.7,9.2,2.5,1.6,13.4,2.5,4.9,0.41,0.34,0.58,1.32,5.77,8.08,11.92,0.30,7.06,5.04,3.77,2.72,1.39,0.88,3.70,4.62,0.54,0.51,0.00,0.00,-0.03,0.0,0.0,0.0,12,10,17,39,170,238,351,8.97,80,41,26,109,136,16,15,0,0,-1,0,0,0,86.30,57.63,1024.10,747.21,1342.33,1028.68,1742.44,1310.56,81.86,57.49,28.67,276.89,313.65,431.88,24.37,1766.6,1485.7,221.4,31.5,5.5,5.7,1.2,-0.3,1.0,6.7,5.4,13.8,2.6,5.0,0.222,0.230,0.326,-0.547,0.033,0.023,0.236,0.214,0.056,0.467,0.089,0.170,2015,0.925,0.925,0,1
1,aaron ekblad,FLA,1996-02-07,19,2014,1.0,1.0,78,1690.82,60.69,56.43,55.41,55.96,57.83,3.29,2.13,31.34,24.20,42.34,34.07,54.45,42.84,2.94,2.14,1.16,7.14,8.27,11.61,0.80,10.51,91.18,0.415,0.132,-0.004,0.359,0.034,-0.022,0.358,0.140,0.011,0.468,0.090,0.174,9.5,3.0,0.0,0.3,0.9,-0.6,9.5,3.4,0.3,13.2,2.5,4.9,0.53,0.28,0.46,1.28,6.46,8.52,11.50,0.36,8.24,6.25,4.25,2.09,1.45,0.85,3.09,4.47,0.67,0.28,0.04,0.04,-0.39,0.0,0.0,0.0,15,8,13,36,182,240,324,10.20,59,41,24,87,126,19,8,1,1,-11,0,0,0,92.85,60.15,883.09,681.85,1193.09,960.11,1534.30,1207.24,82.79,60.38,32.70,201.24,232.98,327.06,22.41,1690.8,1378.1,218.3,55.1,4.2,3.0,0.2,0.2,0.9,4.4,3.2,7.9,1.5,2.9,0.181,0.130,0.056,0.262,0.034,-0.022,0.164,0.135,0.011,0.280,0.054,0.104,2016,0.925,0.925,0,1
2,aaron ekblad,FLA,1996-02-07,20,2014,1.0,1.0,68,1459.28,46.49,55.24,55.40,56.71,53.00,2.21,2.54,35.47,28.75,48.02,38.66,63.21,48.24,2.81,2.49,-0.33,6.72,9.36,14.96,0.32,6.22,91.17,-0.055,-0.007,0.449,0.754,-0.003,0.006,0.013,0.025,0.003,0.037,0.007,0.013,-1.1,-0.1,1.4,0.7,-0.1,0.1,0.3,0.5,0.1,0.9,0.2,0.3,0.41,0.16,0.29,0.86,9.25,12.17,16.78,0.49,4.44,3.38,4.06,2.55,2.38,1.03,3.62,3.45,1.19,0.70,0.00,0.00,-0.49,0.0,0.0,0.0,10,4,7,21,225,296,408,12.03,62,58,25,88,84,29,17,0,0,-12,0,0,0,53.64,61.75,862.68,699.13,1167.80,940.17,1537.28,1173.32,68.38,60.63,-8.11,163.55,227.63,363.96,7.75,1459.3,1194.7,187.3,52.9,-6.2,0.5,0.9,0.7,-0.1,-5.3,1.2,-4.0,-0.8,-1.5,-0.311,0.027,0.275,0.787,-0.003,0.006,-0.231,0.059,0.003,-0.166,-0.031,-0.060,2017,0.925,0.925,0,1
3,aaron ekblad,FLA,1996-02-07,21,2014,1.0,1.0,82,1917.90,53.04,48.15,48.85,48.87,47.70,3.60,3.19,34.88,37.56,47.86,50.11,60.62,63.41,3.26,3.57,0.41,-2.68,-2.25,-2.79,-0.31,10.33,91.51,0.273,-0.197,1.128,-0.540,-0.036,-0.025,0.362,-0.234,-0.061,0.049,0.010,0.018,6.8,-4.9,3.3,-1.6,-1.2,-0.8,10.1,-6.6,-2.0,1.6,0.3,0.6,0.50,0.19,0.50,1.19,5.91,8.85,12.48,0.39,8.47,5.65,4.45,3.79,3.10,1.25,2.16,2.25,0.91,0.28,0.09,0.09,-0.63,0.0,0.0,0.0,16,6,16,38,189,283,399,12.60,121,99,40,69,72,29,9,3,3,-20,0,0,0,115.16,101.95,1114.81,1200.56,1530.00,1601.91,1937.80,2027.05,104.15,114.21,13.21,-85.75,-71.91,-89.25,-10.06,1917.9,1500.0,173.4,181.2,5.1,-3.7,2.6,-1.7,-1.2,7.7,-5.4,0.3,0.1,0.1,0.204,-0.149,0.895,-0.547,-0.036,-0.025,0.276

In [91]:
hockey_df.duplicated().sum()

1

In [92]:
hockey_df[hockey_df.duplicated()]

,Player,Team,Birthday,Age,Draft Yr,Draft Rd,Draft Ov,GP,TOI,GF%,SF%,FF%,CF%,xGF%,GF/60,GA/60,SF/60,SA/60,FF/60,FA/60,CF/60,CA/60,xGF/60,xGA/60,G±/60,S±/60,F±/60,C±/60,xG±/60,oiSh%,Sv%,xEVO_GAR/60,xEVD_GAR/60,xPPO_GAR/60,xSHD_GAR/60,xTake_GAR/60,xDraw_GAR/60,xOff_GAR/60,xDef_GAR/60,xPens_GAR/60,xGAR/60,xWAR/60,xSPAR/60,xEVO_GAR,xEVD_GAR,xPPO_GAR,xSHD_GAR,xTake_GAR,xDraw_GAR,xOff_GAR,xDef_GAR,xPens_GAR,xGAR,xWAR,xSPAR,G/60,A1/60,A2/60,Points/60,iSF/60,iFF/60,iCF/60,ixG/60,Sh%,FSh%,xFSh%,iBLK/60,GIVE/60,TAKE/60,iHF/60,iHA/60,iPENT2/60,iPEND2/60,iPENT5/60,iPEND5/60,iPEN±/60,FOW/60,FOL/60,FO±/60,G,A1,A2,Points,iSF,iFF,iCF,ixG,iBLK,GIVE,TAKE,iHF,iHA,iPENT2,iPEND2,iPENT5,iPEND5,iPEN±,FOW,FOL,FO±,GF,GA,SF,SA,FF,FA,CF,CA,xGF,xGA,G±,S±,F±,C±,xG±,TOI_All,TOI_EV,TOI_PP,TOI_SH,EVO_GAR,EVD_GAR,PPO_GAR,SHD_GAR,Take_GAR,Off_GAR,Def_GAR,GAR,WAR,SPAR,EVO_GAR/60,EVD_GAR/60,PPO_GAR/60,SHD_GAR/60,Take_GAR/60,Draw_GAR/60,Off_GAR/60,Def_GAR/60,Pens_GAR/60,GAR/60,WAR/60,SPAR/60,Year,Cap_Hit,Salary,Left_Handed,Defenceman
2663,jaycob megna,ANA,1992-12-10,25,2012,7.0,210.0,28,406.9,36.96,41.33,42.9,42.58,40.14,1.74,2.96,24.33,34.54,35.32,47.0,46.15,62.25,2.09,3.11,-1.23,-10.21,-11.68,-16.1,-1.02,7.14,91.42,0.032,0.183,0.391,0.002,-0.027,-0.056,0.032,0.158,-0.083,0.101,0.019,0.036,0.2,1.1,0.0,0.0,-0.2,-0.4,0.2,1.1,-0.6,0.7,0.1,0.2,0.15,0.29,0.15,0.59,4.57,6.34,9.58,0.22,3.23,2.33,3.47,4.28,4.42,1.62,4.72,3.98,0.88,0.0,0.0,0.0,-0.88,0.0,0.0,0.0,1,2,1,4,31,43,65,1.49,29,30,11,32,27,6,0,0,0,-6,0,0,0,11.78,20.09,165.0,234.26,239.51,318.74,313.0,422.16,14.15,21.1,-8.31,-69.26,-79.23,-109.16,-6.95,406.9,347.4,0.2,54.9,-0.6,1.2,0.0,-0.6,-0.2,-0.6,0.7,-0.5,-0.1,-0.2,-0.111,0.214,-0.027,-0.61,-0.027,-0.056,-0.111,0.102,-0.083,-0.077,-0.014,-0.027,2019,0.65,0.65,1,1


In [93]:
#It is unclear as to why these rows were duplicated.  However since it is a small number of the overall dataset
#and that this is duplicated data of valid rows that we have already, we will drop these.
hockey_df.drop_duplicates(inplace = True)

/Users/brianjohns/opt/anaconda3/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [94]:
hockey_df.duplicated().sum()

0

In [95]:
hockey_df.head()

,Player,Team,Birthday,Age,Draft Yr,Draft Rd,Draft Ov,GP,TOI,GF%,SF%,FF%,CF%,xGF%,GF/60,GA/60,SF/60,SA/60,FF/60,FA/60,CF/60,CA/60,xGF/60,xGA/60,G±/60,S±/60,F±/60,C±/60,xG±/60,oiSh%,Sv%,xEVO_GAR/60,xEVD_GAR/60,xPPO_GAR/60,xSHD_GAR/60,xTake_GAR/60,xDraw_GAR/60,xOff_GAR/60,xDef_GAR/60,xPens_GAR/60,xGAR/60,xWAR/60,xSPAR/60,xEVO_GAR,xEVD_GAR,xPPO_GAR,xSHD_GAR,xTake_GAR,xDraw_GAR,xOff_GAR,xDef_GAR,xPens_GAR,xGAR,xWAR,xSPAR,G/60,A1/60,A2/60,Points/60,iSF/60,iFF/60,iCF/60,ixG/60,Sh%,FSh%,xFSh%,iBLK/60,GIVE/60,TAKE/60,iHF/60,iHA/60,iPENT2/60,iPEND2/60,iPENT5/60,iPEND5/60,iPEN±/60,FOW/60,FOL/60,FO±/60,G,A1,A2,Points,iSF,iFF,iCF,ixG,iBLK,GIVE,TAKE,iHF,iHA,iPENT2,iPEND2,iPENT5,iPEND5,iPEN±,FOW,FOL,FO±,GF,GA,SF,SA,FF,FA,CF,CA,xGF,xGA,G±,S±,F±,C±,xG±,TOI_All,TOI_EV,TOI_PP,TOI_SH,EVO_GAR,EVD_GAR,PPO_GAR,SHD_GAR,Take_GAR,Off_GAR,Def_GAR,GAR,WAR,SPAR,EVO_GAR/60,EVD_GAR/60,PPO_GAR/60,SHD_GAR/60,Take_GAR/60,Draw_GAR/60,Off_GAR/60,Def_GAR/60,Pens_GAR/60,GAR/60,WAR/60,SPAR/60,Year,Cap_Hit,Salary,Left_Handed,Defenceman
0,aaron ekblad,FLA,1996-02-07,18,2014,1.0,1.0,81,1766.62,59.96,57.82,56.61,57.07,58.74,2.93,1.96,34.78,25.38,45.59,34.94,59.18,44.51,2.78,1.95,0.97,9.40,10.65,14.67,0.83,8.43,92.29,0.183,0.105,1.267,-0.170,0.033,0.023,0.324,0.100,0.056,0.454,0.086,0.165,4.5,2.6,4.7,-0.1,1.0,0.7,9.2,2.5,1.6,13.4,2.5,4.9,0.41,0.34,0.58,1.32,5.77,8.08,11.92,0.30,7.06,5.04,3.77,2.72,1.39,0.88,3.70,4.62,0.54,0.51,0.00,0.00,-0.03,0.0,0.0,0.0,12,10,17,39,170,238,351,8.97,80,41,26,109,136,16,15,0,0,-1,0,0,0,86.30,57.63,1024.10,747.21,1342.33,1028.68,1742.44,1310.56,81.86,57.49,28.67,276.89,313.65,431.88,24.37,1766.6,1485.7,221.4,31.5,5.5,5.7,1.2,-0.3,1.0,6.7,5.4,13.8,2.6,5.0,0.222,0.230,0.326,-0.547,0.033,0.023,0.236,0.214,0.056,0.467,0.089,0.170,2015,0.925,0.925,0,1
1,aaron ekblad,FLA,1996-02-07,19,2014,1.0,1.0,78,1690.82,60.69,56.43,55.41,55.96,57.83,3.29,2.13,31.34,24.20,42.34,34.07,54.45,42.84,2.94,2.14,1.16,7.14,8.27,11.61,0.80,10.51,91.18,0.415,0.132,-0.004,0.359,0.034,-0.022,0.358,0.140,0.011,0.468,0.090,0.174,9.5,3.0,0.0,0.3,0.9,-0.6,9.5,3.4,0.3,13.2,2.5,4.9,0.53,0.28,0.46,1.28,6.46,8.52,11.50,0.36,8.24,6.25,4.25,2.09,1.45,0.85,3.09,4.47,0.67,0.28,0.04,0.04,-0.39,0.0,0.0,0.0,15,8,13,36,182,240,324,10.20,59,41,24,87,126,19,8,1,1,-11,0,0,0,92.85,60.15,883.09,681.85,1193.09,960.11,1534.30,1207.24,82.79,60.38,32.70,201.24,232.98,327.06,22.41,1690.8,1378.1,218.3,55.1,4.2,3.0,0.2,0.2,0.9,4.4,3.2,7.9,1.5,2.9,0.181,0.130,0.056,0.262,0.034,-0.022,0.164,0.135,0.011,0.280,0.054,0.104,2016,0.925,0.925,0,1
2,aaron ekblad,FLA,1996-02-07,20,2014,1.0,1.0,68,1459.28,46.49,55.24,55.40,56.71,53.00,2.21,2.54,35.47,28.75,48.02,38.66,63.21,48.24,2.81,2.49,-0.33,6.72,9.36,14.96,0.32,6.22,91.17,-0.055,-0.007,0.449,0.754,-0.003,0.006,0.013,0.025,0.003,0.037,0.007,0.013,-1.1,-0.1,1.4,0.7,-0.1,0.1,0.3,0.5,0.1,0.9,0.2,0.3,0.41,0.16,0.29,0.86,9.25,12.17,16.78,0.49,4.44,3.38,4.06,2.55,2.38,1.03,3.62,3.45,1.19,0.70,0.00,0.00,-0.49,0.0,0.0,0.0,10,4,7,21,225,296,408,12.03,62,58,25,88,84,29,17,0,0,-12,0,0,0,53.64,61.75,862.68,699.13,1167.80,940.17,1537.28,1173.32,68.38,60.63,-8.11,163.55,227.63,363.96,7.75,1459.3,1194.7,187.3,52.9,-6.2,0.5,0.9,0.7,-0.1,-5.3,1.2,-4.0,-0.8,-1.5,-0.311,0.027,0.275,0.787,-0.003,0.006,-0.231,0.059,0.003,-0.166,-0.031,-0.060,2017,0.925,0.925,0,1
3,aaron ekblad,FLA,1996-02-07,21,2014,1.0,1.0,82,1917.90,53.04,48.15,48.85,48.87,47.70,3.60,3.19,34.88,37.56,47.86,50.11,60.62,63.41,3.26,3.57,0.41,-2.68,-2.25,-2.79,-0.31,10.33,91.51,0.273,-0.197,1.128,-0.540,-0.036,-0.025,0.362,-0.234,-0.061,0.049,0.010,0.018,6.8,-4.9,3.3,-1.6,-1.2,-0.8,10.1,-6.6,-2.0,1.6,0.3,0.6,0.50,0.19,0.50,1.19,5.91,8.85,12.48,0.39,8.47,5.65,4.45,3.79,3.10,1.25,2.16,2.25,0.91,0.28,0.09,0.09,-0.63,0.0,0.0,0.0,16,6,16,38,189,283,399,12.60,121,99,40,69,72,29,9,3,3,-20,0,0,0,115.16,101.95,1114.81,1200.56,1530.00,1601.91,1937.80,2027.05,104.15,114.21,13.21,-85.75,-71.91,-89.25,-10.06,1917.9,1500.0,173.4,181.2,5.1,-3.7,2.6,-1.7,-1.2,7.7,-5.4,0.3,0.1,0.1,0.204,-0.149,0.895,-0.547,-0.036,-0.025,0.276

In [96]:
hockey_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6226 entries, 0 to 6347
Columns: 146 entries, Player to Defenceman
dtypes: datetime64[ns](1), float64(117), int64(26), object(2)
memory usage: 7.0+ MB


In [97]:
hockey_df.shape

(6226, 146)

In [98]:
#Prior to pushing through on our modeling, I am going to create a unique ID that reflects the Player-Age-Season
#This could be used for indexing if needed
#Here we will create the new column that will include that information
hockey_df['ID'] = hockey_df['Player']+'-'+hockey_df['Age'].astype('str')+'-'+hockey_df['Year'].astype('str')

/var/folders/dm/5cg2631s7057bxm5mx52z5fr0000gn/T/ipykernel_71194/2831129813.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hockey_df['ID'] = hockey_df['Player']+'-'+hockey_df['Age'].astype('str')+'-'+hockey_df['Year'].astype('str')


In [99]:
hockey_df.head()

,Player,Team,Birthday,Age,Draft Yr,Draft Rd,Draft Ov,GP,TOI,GF%,SF%,FF%,CF%,xGF%,GF/60,GA/60,SF/60,SA/60,FF/60,FA/60,CF/60,CA/60,xGF/60,xGA/60,G±/60,S±/60,F±/60,C±/60,xG±/60,oiSh%,Sv%,xEVO_GAR/60,xEVD_GAR/60,xPPO_GAR/60,xSHD_GAR/60,xTake_GAR/60,xDraw_GAR/60,xOff_GAR/60,xDef_GAR/60,xPens_GAR/60,xGAR/60,xWAR/60,xSPAR/60,xEVO_GAR,xEVD_GAR,xPPO_GAR,xSHD_GAR,xTake_GAR,xDraw_GAR,xOff_GAR,xDef_GAR,xPens_GAR,xGAR,xWAR,xSPAR,G/60,A1/60,A2/60,Points/60,iSF/60,iFF/60,iCF/60,ixG/60,Sh%,FSh%,xFSh%,iBLK/60,GIVE/60,TAKE/60,iHF/60,iHA/60,iPENT2/60,iPEND2/60,iPENT5/60,iPEND5/60,iPEN±/60,FOW/60,FOL/60,FO±/60,G,A1,A2,Points,iSF,iFF,iCF,ixG,iBLK,GIVE,TAKE,iHF,iHA,iPENT2,iPEND2,iPENT5,iPEND5,iPEN±,FOW,FOL,FO±,GF,GA,SF,SA,FF,FA,CF,CA,xGF,xGA,G±,S±,F±,C±,xG±,TOI_All,TOI_EV,TOI_PP,TOI_SH,EVO_GAR,EVD_GAR,PPO_GAR,SHD_GAR,Take_GAR,Off_GAR,Def_GAR,GAR,WAR,SPAR,EVO_GAR/60,EVD_GAR/60,PPO_GAR/60,SHD_GAR/60,Take_GAR/60,Draw_GAR/60,Off_GAR/60,Def_GAR/60,Pens_GAR/60,GAR/60,WAR/60,SPAR/60,Year,Cap_Hit,Salary,Left_Handed,Defenceman,ID
0,aaron ekblad,FLA,1996-02-07,18,2014,1.0,1.0,81,1766.62,59.96,57.82,56.61,57.07,58.74,2.93,1.96,34.78,25.38,45.59,34.94,59.18,44.51,2.78,1.95,0.97,9.40,10.65,14.67,0.83,8.43,92.29,0.183,0.105,1.267,-0.170,0.033,0.023,0.324,0.100,0.056,0.454,0.086,0.165,4.5,2.6,4.7,-0.1,1.0,0.7,9.2,2.5,1.6,13.4,2.5,4.9,0.41,0.34,0.58,1.32,5.77,8.08,11.92,0.30,7.06,5.04,3.77,2.72,1.39,0.88,3.70,4.62,0.54,0.51,0.00,0.00,-0.03,0.0,0.0,0.0,12,10,17,39,170,238,351,8.97,80,41,26,109,136,16,15,0,0,-1,0,0,0,86.30,57.63,1024.10,747.21,1342.33,1028.68,1742.44,1310.56,81.86,57.49,28.67,276.89,313.65,431.88,24.37,1766.6,1485.7,221.4,31.5,5.5,5.7,1.2,-0.3,1.0,6.7,5.4,13.8,2.6,5.0,0.222,0.230,0.326,-0.547,0.033,0.023,0.236,0.214,0.056,0.467,0.089,0.170,2015,0.925,0.925,0,1,aaron ekblad-18-2015
1,aaron ekblad,FLA,1996-02-07,19,2014,1.0,1.0,78,1690.82,60.69,56.43,55.41,55.96,57.83,3.29,2.13,31.34,24.20,42.34,34.07,54.45,42.84,2.94,2.14,1.16,7.14,8.27,11.61,0.80,10.51,91.18,0.415,0.132,-0.004,0.359,0.034,-0.022,0.358,0.140,0.011,0.468,0.090,0.174,9.5,3.0,0.0,0.3,0.9,-0.6,9.5,3.4,0.3,13.2,2.5,4.9,0.53,0.28,0.46,1.28,6.46,8.52,11.50,0.36,8.24,6.25,4.25,2.09,1.45,0.85,3.09,4.47,0.67,0.28,0.04,0.04,-0.39,0.0,0.0,0.0,15,8,13,36,182,240,324,10.20,59,41,24,87,126,19,8,1,1,-11,0,0,0,92.85,60.15,883.09,681.85,1193.09,960.11,1534.30,1207.24,82.79,60.38,32.70,201.24,232.98,327.06,22.41,1690.8,1378.1,218.3,55.1,4.2,3.0,0.2,0.2,0.9,4.4,3.2,7.9,1.5,2.9,0.181,0.130,0.056,0.262,0.034,-0.022,0.164,0.135,0.011,0.280,0.054,0.104,2016,0.925,0.925,0,1,aaron ekblad-19-2016
2,aaron ekblad,FLA,1996-02-07,20,2014,1.0,1.0,68,1459.28,46.49,55.24,55.40,56.71,53.00,2.21,2.54,35.47,28.75,48.02,38.66,63.21,48.24,2.81,2.49,-0.33,6.72,9.36,14.96,0.32,6.22,91.17,-0.055,-0.007,0.449,0.754,-0.003,0.006,0.013,0.025,0.003,0.037,0.007,0.013,-1.1,-0.1,1.4,0.7,-0.1,0.1,0.3,0.5,0.1,0.9,0.2,0.3,0.41,0.16,0.29,0.86,9.25,12.17,16.78,0.49,4.44,3.38,4.06,2.55,2.38,1.03,3.62,3.45,1.19,0.70,0.00,0.00,-0.49,0.0,0.0,0.0,10,4,7,21,225,296,408,12.03,62,58,25,88,84,29,17,0,0,-12,0,0,0,53.64,61.75,862.68,699.13,1167.80,940.17,1537.28,1173.32,68.38,60.63,-8.11,163.55,227.63,363.96,7.75,1459.3,1194.7,187.3,52.9,-6.2,0.5,0.9,0.7,-0.1,-5.3,1.2,-4.0,-0.8,-1.5,-0.311,0.027,0.275,0.787,-0.003,0.006,-0.231,0.059,0.003,-0.166,-0.031,-0.060,2017,0.925,0.925,0,1,aaron ekblad-20-2017
3,aaron ekblad,FLA,1996-02-07,21,2014,1.0,1.0,82,1917.90,53.04,48.15,48.85,48.87,47.70,3.60,3.19,34.88,37.56,47.86,50.11,60.62,63.41,3.26,3.57,0.41,-2.68,-2.25,-2.79,-0.31,10.33,91.51,0.273,-0.197,1.128,-0.540,-0.036,-0.025,0.362,-0.234,-0.061,0.049,0.010,0.018,6.8,-4.9,3.3,-1.6,-1.2,-0.8,10.1,-6.6,-2.0,1.6,0.3,0.6,0.50,0.19,0.50,1.19,5.91,8.85,12.48,0.39,8.47,5.65,4.45,3.79,3.10,1.25,2.16,2.25,0.91,0.28,0.09,0.09,-0.63,0.0,0.0,0.0,16,6,16,38,189,283,399,12.60,121,99,40,69,72,29,9,3,3,-20,0,0,0,115.16,101.95,1114.81,1200.56,1530.00,1601.91,1937.80,2027.05,104.15,114.21,13.21,-85.75,-71.91,-89.25,-10.06,1917.9,1500.0,173.4,181.2,5.1,-3.7,2.6,-1.7,-1.2,

With this, the cleaning of our data is complete and ready for use in modeling.

Considering that this notebook scrapes the most up-to-date information from the CapFriendly website, the specific contents of this data would change if re-run.

I made my final webscrape on March 25, and dated the outgoing data into a csv.  This specific .csv file will be included with my submission in order for it to be read into the next notebook and give consistent results for my Regression and Clustering models.

In [100]:
hockey_df.to_csv('data/CLEAN_CAPSTONE.csv',index=False)

### Next Steps

1. The csv output from this notebook will be placed in a data folder.  However, this is NOT the data that will be used for the next notebook.
2. Ensure that the 'CLEAN_CAPSTONE_250322.csv' that has been provided is in the **SAME** folder as where the notebook is running.
3. The next notebook is titled **Brian Johns Capstone #2 - Regression Modeling**